In [19]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("OPEN_API_KEY")

In [20]:
llm_config_mini = {"model": "gpt-4o-mini"}
llm_config_large = {"model": "gpt-4o"}


# Test Simple Autogen Agent Reply

In [26]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config_mini,
    human_input_mode="NEVER",
)

[autogen.oai.client: 11-02 11:38:37] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [27]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

Why don’t skeletons fight each other?

They don’t have the guts!


# Sequential Chat with Simple Entity Extractor Agent

## Base Agent

In [31]:
entity_extraction_agent = ConversableAgent(
    name="Entity_Extraction_Agent",
    system_message='''You are an Entity Extraction Agent,
    You are an AI Agent tasked with extracting entitites from SAR Narratives.
    
    Using the SAR narrative shared by the user, do the following:

    Step 1) Extract the main entities described. Entities could be Individual or Organizations or Financial Institutions 

    Step 2) Extract the account IDs described. Account IDs could be numeric or alphanumeric. If an Account ID is missing, create a dummy account ID with the prefix "Dummy_".

    Step 3) Map the extracted account IDs to Individuals or Organizations. Return this in the form of a dictionary, E.g. {"Entity_Name": ["Account_ID"]}

    Step 4) Map the extracted account IDs to Financial institutions where the account is held. Return this in the form of a dictionary, E.g. {"Financial Instituion": ["Account_ID"]}

    An example is given below.

    Narrative:

    John deposited $5000 in Cash into Acct #345723 at Bank of America. John sends $3000 to Jill's account at  Chase. Jill deposited $3000 in Cash into her Acct at Chase Bank.
    John and Jill own a business Acme Inc that has a  Business account, Account #98765 . John sends $2000 from Acct #345723 to Account #98765. Jill sends $1000 from 
    her Acct at Chase Bank to Acct #98765.

    Step 1) Extract the main entities described. Identify the Individuals , Organizations and Financial Instituions mentioned. 
    Individuals and Organizations are entities that usually conduct transactions through Financial institutions. Here, John and Jill are Individuals. Acme Inc is an Organization.
    Financial institutions are institutions such as banks or insurance companies that offer financial services to customers. Hence Bank of America and Chase Bank are Financial institutions.

    Record this as a dictionary named Entities.


    Entities = {"Individuals": ["John", "Jill"],"Organizations":["Acme Inc"],"Financial Institutions":["Chase"," Bank of America"]}}

    Step 2) Extract the account IDs described. Accounts mentioned in the narrative are #345723,#98765. There is also an account at Chase Bank that is missing. Assign a dummy account ID ("Dummy_12345")
    to this account.

    Record this in a list named Account_IDs
    Account_IDs = ["345723","98765","Dummy_12345"]

    Step 3) Map the Account IDs extracted in Step 2 to Financial Instititions where they are held. Account ID #345723 is held at bank of America.Account ID Dummy_12345 is held at Chase Bank.
    Account ID #98765 is referenced but the Financial instituion where it is held is not specified. So it can be assumed to be held at "Dummy_Bank"

    Record this in a dictionary named FIs_to_Accts.
    FIs_to_Accts = {"Bank of America":["345723"],"Chase Bank":["Dummy_12345"],"Dummy_Bank":"98765"}

    
    Step 4) Map the extracted Account IDs extracted in Step 2 to Individuals or Organizations that are customers of the Financial Institutions. Account ID #345723 belongs to John. 
    Account ID #98765 belongs to Acme Inc. 
    Account ID Dummy_12345 belongs to Jill.
    

    Record this in a dictionary named Customers_to_Accts

    Customers_to_Accts = {"John":["345723"],"Jill":["Dummy_12345"],"Acme Inc":"98765"}

    
    
    ''',
    llm_config=llm_config_mini,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 12-29 15:40:08] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [32]:
user_agent = ConversableAgent(
    name="user_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS"
)

In [7]:
chats = [
    {
        "sender": entity_extraction_agent,
        "recipient": user_agent,
        "message": 
            "Hello, I'm here to help you extract entities from the SAR narrative. "
            "Please enter the SAR narrative",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the extracted entity information s JSON object only: "
                             "{'Individuals':['Entity1','Entity2'],'Organizations':['Entity3'],'Financial_Institutions':['Entity4','Entity5']}"
                             "{'Account IDs': ['Acct_ID_1','Acct_ID_2','Acct_ID_3']}"
                             "{'FIs_to_Accts': { 'Entity4': ['Acct_ID_1'],'Entity5':['Acct_ID_2','Acct_ID_3]} }"
                             "{'Customers_to_Accts': { 'Entity1': ['Acct_ID_1'],'Entity2':['Acct_ID_2'],'Entity3':['Acct_ID_3']} }                                                                      ",
        },
        "max_turns": 1,
        "clear_history" : False
    },
]

In [105]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Entity Extraction Agent (to user_agent):

Hello, I'm here to help you extract entities from the SAR narrative. Please enter the SAR narrative

--------------------------------------------------------------------------------
user_agent (to Entity Extraction Agent):

This SAR is being filed to summarize suspicious cash deposits and wire transfer activity conducted by John Doe, account #12345678910.  John  Doe has been a bank customer since April 2000.  Mr. Doe is a college  student and employed part-time at Quickie Car Wash.    Cash deposits to Mr. Doe’s personal checking account are structured to possibly circumvent federal reporting requirements.  The deposits are  followed by immediate wire transfers to Aussie Bank in Sydney, Australia to a single beneficiary, Jennifer Doe, account #981012345, with a

In [106]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

```json
{
  "Individuals": [
    "John Doe",
    "Jennifer Doe"
  ],
  "Organizations": [
    "Quickie Car Wash"
  ],
  "Financial_Institutions": [
    "Aussie Bank",
    "Big City, FL"
  ],
  "Account IDs": [
    "12345678910",
    "981012345"
  ],
  "FIs_to_Accts": {
    "Aussie Bank": [
      "981012345"
    ],
    "Big City, FL": [
      "12345678910"
    ]
  },
  "Customers_to_Accts": {
    "John Doe": [
      "12345678910"
    ],
    "Jennifer Doe": [
      "981012345"
    ]
  }
}
```




## Try without explicit user input

In [33]:
sar3 = '''
This SAR is being filed to summarize suspicious cash deposits and wire transfer activity conducted by John Doe, account #12345678910.  John  Doe has been a bank customer since April 2000.  Mr. Doe is a college  student and employed part-time at Quickie Car Wash.  

Cash deposits to Mr. Doe’s personal checking account are structured to possibly circumvent federal reporting requirements.  The deposits are  followed by immediate wire transfers to Aussie Bank in Sydney, Australia to a single beneficiary, Jennifer Doe, account #981012345, with an address located in Australia.  Specifically the following activity has been 
observed: cash deposits (dates followed by amounts): 03/15/02  $9,950.00; 03/17/02 $9,700.00; 03/18/02 $10,000; total: $29,650.  Wire transfers out (dates followed by amounts): 03/16/02 $9,900.00, 03/18/02  $9,700.00, 03/19/02 $9,900.00.  

The volume and frequency of the deposits is not consistent with previous banking transactions conducted by Mr. Doe.  The amounts of currency do not appear consistent with the 
customer’s stated employment.  Also, the relationship between the customer and Jennifer Doe and the purpose for the wire activity is unknown. 
Therefore, due to the structured cash deposits by the customer on almost consecutive days into the account, and the immediate wire transfer of the funds out of the account to Jennifer Doe, Aussie bank, account  #981012345, Sydney Australia, this SAR is being filed.  Investigation is continuing. 
The bank’s financial intelligence unit in Big City, FL, maintains all records related to this SAR

'''
message = f'''
 Please analyze the following supicious activity report:
 {sar3}

'''

In [34]:
user_agent = ConversableAgent(
    name="user_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="NEVER"
)

In [35]:
chats = [
    {
        "sender": user_agent,
        "recipient": entity_extraction_agent,
        "message": message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the extracted entity information s JSON object only: "
                             "{'Individuals':['Entity1','Entity2'],'Organizations':['Entity3'],'Financial_Institutions':['Entity4','Entity5']}"
                             "{'Account IDs': ['Acct_ID_1','Acct_ID_2','Acct_ID_3']}"
                             "{'FIs_to_Accts': { 'Entity4': ['Acct_ID_1'],'Entity5':['Acct_ID_2','Acct_ID_3]} }"
                             "{'Customers_to_Accts': { 'Entity1': ['Acct_ID_1'],'Entity2':['Acct_ID_2'],'Entity3':['Acct_ID_3']} }                                                                      ",
        },
        "max_turns": 1,
        "clear_history" : False
    },
]

In [36]:
chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
user_agent (to Entity_Extraction_Agent):


 Please analyze the following supicious activity report:
 
This SAR is being filed to summarize suspicious cash deposits and wire transfer activity conducted by John Doe, account #12345678910.  John  Doe has been a bank customer since April 2000.  Mr. Doe is a college  student and employed part-time at Quickie Car Wash.  

Cash deposits to Mr. Doe’s personal checking account are structured to possibly circumvent federal reporting requirements.  The deposits are  followed by immediate wire transfers to Aussie Bank in Sydney, Australia to a single beneficiary, Jennifer Doe, account #981012345, with an address located in Australia.  Specifically the following activity has been 
observed: cash deposits (dates followed by amounts): 03/15/02  $9,950.00; 03/17/02 $9

In [37]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

```json
{
  "Individuals": ["John Doe", "Jennifer Doe"],
  "Organizations": ["Quickie Car Wash"],
  "Financial_Institutions": ["Aussie Bank"],
  "Account IDs": ["12345678910", "981012345"],
  "FIs_to_Accts": {
    "Dummy_Bank": ["12345678910"],
    "Aussie Bank": ["981012345"]
  },
  "Customers_to_Accts": {
    "John Doe": ["12345678910"],
    "Jennifer Doe": ["981012345"]
  }
}
```




In [41]:
results = chat_results[0].summary.split('```',2)[1]
results = results.strip('```json').strip('```')

In [43]:
import json
json.loads(results)

{'Individuals': ['John Doe', 'Jennifer Doe'],
 'Organizations': ['Quickie Car Wash'],
 'Financial_Institutions': ['Aussie Bank'],
 'Account IDs': ['12345678910', '981012345'],
 'FIs_to_Accts': {'Dummy_Bank': ['12345678910'], 'Aussie Bank': ['981012345']},
 'Customers_to_Accts': {'John Doe': ['12345678910'],
  'Jennifer Doe': ['981012345']}}

## Change Mapping to make Account ID the Key - Final Prompt

In [5]:
from autogen import ConversableAgent

In [6]:
entity_extraction_agent = ConversableAgent(
    name="Entity_Extraction_Agent",
    system_message='''You are an Entity Extraction Agent, an AI Assistant tasked with extracting entitites from SAR Narratives.
    
    Using the SAR narrative shared by the user, do the following:

    Step 1) Extract the main entities described. Entities could be Individuals or Organizations or Financial Institutions. Return this in the form of a dictionary called Entities
            Entities = { "Individuals" : <List of Individuals> , "Organizations": <List of Organizations>, "Financial_Institutions": <List of Financial Instituions >} 

    Step 2) Extract the Account IDs described. Account IDs could be numeric or alphanumeric.If an Account ID is missing, create a Dummy account ID with the prefix "Dummy_".
            e.g. Dummy_Acct_1, Dummy_Acct_2 and so on. Put this in a list called Account_IDs

    Step 3) Map the extracted Account IDs to Financial institutions where the account is held. If the name of the institution is not specified, create a Dummy institution name e.g Dummy_Bank_1, Dummy_Bank_2 etc.
            Return this in the form of a dictionary  called Acct_to_FI E.g. {<Account_ID> : <Financial Instituion>}

    Step 4) Map the extracted account IDs to Individuals or Organizations who own the account. Return this in the form of a dictionary called Acct_to_Cust E.g. {<Account_ID>: <Entity_Name>}

    

    An example is given below.

    Narrative:

    John deposited $5000 in Cash into Acct #345723 at Bank of America. John sends $3000 to Jill's account at  Chase. Jill deposited $3000 in Cash into her Acct at Chase Bank.
    John and Jill own a business Acme Inc that has a  Business account, Account #98765 . John sends $2000 from Acct #345723 to Account #98765. Jill sends $1000 from 
    her Acct at Chase Bank to Acct #98765.

    Step 1) Extract the main entities described. Identify the Individuals , Organizations and Financial Instituions mentioned. 
    Individuals and Organizations are entities that usually conduct transactions through Financial institutions. Here, John and Jill are Individuals. Acme Inc is an Organization.
    Financial institutions are institutions such as banks or insurance companies that offer financial services to customers. Hence Bank of America and Chase Bank are Financial institutions.

    Record this as a dictionary named Entities.

    Entities = {"Individuals": ["John", "Jill"],"Organizations":["Acme Inc"],"Financial Institutions":["Chase"," Bank of America"]}}

    Step 2) Extract the account IDs described. Accounts mentioned in the narrative are #345723,#98765. There is also an account at Chase Bank that is missing. Assign a dummy account ID ("Dummy_Acct_1")
    to this account.

    Record this in a list named Account_IDs
    Account_IDs = ["345723","98765","Dummy_Acct_1"]

    Step 3) Map the Account IDs extracted in Step 2 to Financial Instititions where they are held. Account ID #345723 is held at Bank of America. Account ID Dummy_Acct_1 is held at Chase Bank.
    Account ID #98765 is referenced but the Financial instituion where it is held is not specified. So it can be assumed to be held at "Dummy_Bank_1".

    Multiple Accounts could be held at the same Financial Institution.

    Record this in a dictionary named FI_to_Acct.
    FI_to_Acct = {"Bank of America":["345723"], "Chase Bank" : ["Dummy_Acct_1"], "Dummy_Bank_1": ["98765"]  }

    
    Step 4) Map the extracted Account IDs extracted in Step 2 to Individuals or Organizations that are customers of the Financial Institutions. 
    Account ID #345723 belongs to John. 
    Account ID #98765 belongs to Acme Inc. 
    Account ID Dummy_Acct_1 belongs to Jill.
    

    Record this in a dictionary named Acct_to_Cust

    Acct_to_Cust = {"345723": "John,"Dummy_Acct_1" : "Jill","98765": "Acme Inc"}
    
    
    ''',
    llm_config=llm_config_mini,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 11-16 15:36:19] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [12]:
chats = [
    {
        "sender": user_agent,
        "recipient": entity_extraction_agent,
        "message": message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the extracted entity information s JSON object only: "
                             "{'Entites': {'Individuals':['Individual_1','Individual_2'],'Organizations':['Organization_1'],'Financial_Institutions':['FI_1','FI_2']}"
                             " 'Account IDs': ['Acct_ID_1','Acct_ID_2','Acct_ID_3']"
                             " 'FI_to_Acct' : {'FI_1': ['Acct_ID_1'],'FI_2': ['Acct_ID_2','Acct_ID_3']}"
                             " 'Acct_to_Cust' : { 'Acct_ID_1': 'Individual_1','Acct_ID_2': 'Individual_2','Acct_ID_3': Organization_1} }"                                                                      
        },
        "max_turns": 1,
        "clear_history" : False
    },
]

In [13]:
from autogen import initiate_chats
chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
user_agent (to Entity_Extraction_Agent):


 Please analyze the following supicious activity report:
 
This SAR is being filed to summarize suspicious cash deposits and wire transfer activity conducted by John Doe, account #12345678910.  John  Doe has been a bank customer since April 2000.  Mr. Doe is a college  student and employed part-time at Quickie Car Wash.  

Cash deposits to Mr. Doe’s personal checking account are structured to possibly circumvent federal reporting requirements.  The deposits are  followed by immediate wire transfers to Aussie Bank in Sydney, Australia to a single beneficiary, Jennifer Doe, account #981012345, with an address located in Australia.  Specifically the following activity has been 
observed: cash deposits (dates followed by amounts): 03/15/02  $9,950.00; 03/17/02 $9

In [14]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

```json
{
    "Entities": {
        "Individuals": ["John Doe", "Jennifer Doe"],
        "Organizations": ["Quickie Car Wash", "Aussie Bank"],
        "Financial_Institutions": []
    },
    "Account_IDs": ["12345678910", "981012345"],
    "FI_to_Acct": {
        "Dummy_Bank_1": ["12345678910"],
        "Aussie Bank": ["981012345"]
    },
    "Acct_to_Cust": {
        "12345678910": "John Doe",
        "981012345": "Jennifer Doe"
    }
}
```




## Test with Last Msg instead of Reflection

Try using `last_msg`

In [107]:
entity_extraction_agent = ConversableAgent(
    name="Entity Extraction Agent",
    system_message='''You are an Entity Extraction Agent,
    You are an AI Agent tasked with extracting entitites from SAR Narratives.
    
    Using the SAR narrative shared by the user, do the following:

    Step 1) Extract the main entities described. Entities could be Individual or Organizations or Financial Institutions 

    Step 2) Extract the account IDs described. Account IDs could be numeric or alphanumeric. If an Account ID is missing, create a dummy account ID with the prefix "Dummy_".

    Step 3) Map the extracted account IDs to Individuals or Organizations. Return this in the form of a dictionary, E.g. {"Entity_Name": ["Account_ID"]}

    Step 4) Map the extracted account IDs to Financial institutions where the account is held. Return this in the form of a dictionary, E.g. {"Financial Instituion": ["Account_ID"]}

    An example is given below.

    Narrative:

    John deposited $5000 in Cash into Acct #345723 at Bank of America. John sends $3000 to Jill's account at  Chase. Jill deposited $3000 in Cash into her Acct at Chase Bank.
    John and Jill own a business Acme Inc that has a  Business account, Account #98765 . John sends $2000 from Acct #345723 to Account #98765. Jill sends $1000 from 
    her Acct at Chase Bank to Acct #98765.

    Step 1) Extract the main entities described. Identify the Individuals , Organizations and Financial Instituions mentioned. 
    Individuals and Organizations are entities that usually conduct transactions through Financial institutions. Here, John and Jill are Individuals. Acme Inc is an Organization.
    Financial institutions are institutions such as banks or insurance companies that offer financial services to customers. Hence Bank of America and Chase Bank are Financial institutions.

    Record this as a dictionary named Entities.


    Entities = {"Individuals": ["John", "Jill"],"Organizations":["Acme Inc"],"Financial Institutions":["Chase"," Bank of America"]}}

    Step 2) Extract the account IDs described. Accounts mentioned in the narrative are #345723,#98765. There is also an account at Chase Bank that is missing. Assign a dummy account ID ("Dummy_12345")
    to this account.

    Record this in a list named Account_IDs
    Account_IDs = ["345723","98765","Dummy_12345"]

    Step 3) Map the Account IDs extracted in Step 2 to Financial Instititions where they are held. Account ID #345723 is held at bank of America.Account ID Dummy_12345 is held at Chase Bank.
    Account ID #98765 is referenced but the Financial instituion where it is held is not specified. So it can be assumed to be held at "Dummy_Bank"

    Record this in a dictionary named FIs_to_Accts.
    FIs_to_Accts = {"Bank of America":["345723"],"Chase Bank":["Dummy_12345"],"Dummy_Bank":"98765"}

    
    Step 4) Map the extracted Account IDs extracted in Step 2 to Individuals or Organizations that are customers of the Financial Institutions. Account ID #345723 belongs to John. 
    Account ID #98765 belongs to Acme Inc. 
    Account ID Dummy_12345 belongs to Jill.
    

    Record this in  a dictionary named Customers_to_Accts

    Customers_to_Accts = {"John":["345723"],"Jill":["Dummy_12345"],"Acme Inc":"98765"}

    
    
    ''',
    llm_config=llm_config_mini,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 09-02 08:42:48] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [108]:
user_agent = ConversableAgent(
    name="user_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "Account IDs" in msg.get("content").lower(),
)

In [112]:
chats = [
    {
        "sender": entity_extraction_agent,
        "recipient": user_agent,
        "message": 
            "Hello, I'm here to help you extract entities from the SAR narrative. "
            "Please enter the SAR narrative",
        "summary_method": "last_msg",
        "summary_args": {
            "summary_prompt" : "Return the extracted entity information s JSON object only: "
                             "{'Individuals':['Entity1','Entity2'],'Organizations':['Entity3'],'Financial_Institutions':['Entity4','Entity5']}"
                             "{'Account IDs': ['Acct_ID_1','Acct_ID_2','Acct_ID_3']}"
                             "{'FIs_to_Accts': { 'Entity4': ['Acct_ID_1'],'Entity5':['Acct_ID_2','Acct_ID_3]} }"
                             "{'Customers_to_Accts': { 'Entity1': ['Acct_ID_1'],'Entity2':['Acct_ID_2'],'Entity3':['Acct_ID_3']} }                                                                      ",
        },
        "max_turns": 2,
        "clear_history" : True
    },
]

In [113]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Entity Extraction Agent (to user_agent):

Hello, I'm here to help you extract entities from the SAR narrative. Please enter the SAR narrative

--------------------------------------------------------------------------------
user_agent (to Entity Extraction Agent):

This SAR is being filed to summarize suspicious cash deposits and wire transfer activity conducted by John Doe, account #12345678910.  John  Doe has been a bank customer since April 2000.  Mr. Doe is a college  student and employed part-time at Quickie Car Wash.    Cash deposits to Mr. Doe’s personal checking account are structured to possibly circumvent federal reporting requirements.  The deposits are  followed by immediate wire transfers to Aussie Bank in Sydney, Australia to a single beneficiary, Jennifer Doe, account #981012345, with a

In [111]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

This SAR is being filed to summarize suspicious cash deposits and wire transfer activity conducted by John Doe, account #12345678910.  John  Doe has been a bank customer since April 2000.  Mr. Doe is a college  student and employed part-time at Quickie Car Wash.    Cash deposits to Mr. Doe’s personal checking account are structured to possibly circumvent federal reporting requirements.  The deposits are  followed by immediate wire transfers to Aussie Bank in Sydney, Australia to a single beneficiary, Jennifer Doe, account #981012345, with an address located in Australia.  Specifically the following activity has been  observed: cash deposits (dates followed by amounts): 03/15/02  $9,950.00; 03/17/02 $9,700.00; 03/18/02 $10,000; total: $29,650.  Wire transfers out (dates followed by amounts): 03/16/02 $9,900.00, 03/18/02  $9,700.00, 03/19/02 $9,900.00.    The volume and frequency of the deposits is not consistent with previous banking transactions conducted by Mr. Doe.  The amounts of cu

# Sequential Chat with Entity Extraction Agent + Reflection with Critic Agent

In [14]:
sar3 = '''
This SAR is being filed to summarize suspicious cash deposits and wire transfer activity conducted by John Doe, account #12345678910.  John  Doe has been a bank customer since April 2000.  Mr. Doe is a college  student and employed part-time at Quickie Car Wash.  

Cash deposits to Mr. Doe’s personal checking account are structured to possibly circumvent federal reporting requirements.  The deposits are  followed by immediate wire transfers to Aussie Bank in Sydney, Australia to a single beneficiary, Jennifer Doe, account #981012345, with an address located in Australia.  Specifically the following activity has been 
observed: cash deposits (dates followed by amounts): 03/15/02  $9,950.00; 03/17/02 $9,700.00; 03/18/02 $10,000; total: $29,650.  Wire transfers out (dates followed by amounts): 03/16/02 $9,900.00, 03/18/02  $9,700.00, 03/19/02 $9,900.00.  

The volume and frequency of the deposits is not consistent with previous banking transactions conducted by Mr. Doe.  The amounts of currency do not appear consistent with the 
customer’s stated employment.  Also, the relationship between the customer and Jennifer Doe and the purpose for the wire activity is unknown. 
Therefore, due to the structured cash deposits by the customer on almost consecutive days into the account, and the immediate wire transfer of the funds out of the account to Jennifer Doe, Aussie bank, account  #981012345, Sydney Australia, this SAR is being filed.  Investigation is continuing. 
The bank’s financial intelligence unit in Big City, FL, maintains all records related to this SAR

'''
message = f'''
 Please analyze the following supicious activity report:
 {sar3}

'''

In [11]:
sar_agent = ConversableAgent(
    name="SAR_Agent",
    system_message="You are assigned a SAR from which to extract transactions. You do this by sharing this SAR with other assistant agents who "
                   "work on the SAR and extract useful information.",
    is_termination_msg = lambda x: x.get("content","") and x.get("content", "").find("SATISFACTORY")>=0,
    llm_config=llm_config_mini,
    max_consecutive_auto_reply=5,
    human_input_mode="NEVER",
)

[autogen.oai.client: 10-28 21:23:34] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [9]:
# Reflection Agent for entity extraction
reflection_agent_ee  = ConversableAgent(
    name = "reflection_agent_ee",
    system_message ='''
                        You receive the following information to review:
                            "Account IDs" : List of Account IDs mentioned in the Suspicious Aactivity Repot (SAR)
                            "FIs_to_Accts" : Mapping between above Account IDs and Financial instituions(FI) they belong to. If the name of the FI is not explicitly mentioned, it will
                                             be identified as a Dummy_Bank
                            "Customers_to_Accts": Mapping between customer names and accounts they own.


                        You will review this information for accuracy in the context of the provided SAR and provide feedback if any information is missing or incorrect.
                        If the information is satisfactory, just return the word SATISFACTORY.

                    ''',
    llm_config = llm_config_mini
)

[autogen.oai.client: 10-27 13:09:05] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [28]:
def reflection_message(recipient,sar,sender,config):
    print ("Reflecting...")
    return f''' Review the information below in the context of the SAR narrative provided below and provide any feedback for improvement \n
             
                INFORMATION TO REVIEW: \n {recipient.chat_messages_for_summary(sender)[-1]['content']}
                SAR NARATIVE: \n {sar}

            '''

In [19]:

nested_chat_queue = [
    {
        "recipient": reflection_agent_ee,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {"summary_prompt" : 
            "Return review into as JSON object only:"
            "{'Review': < Review >} ",},
        "max_turns": 1,
    },
]

In [20]:
sar_agent.register_nested_chats(
    nested_chat_queue,
    trigger=entity_extraction_agent,
    # position=4,
)

In [21]:

res = sar_agent.initiate_chat(
    recipient=entity_extraction_agent,
    message=sar3,
    max_turns=2,
    summary_method="reflection_with_llm",
    summary_args = {
                 "summary_prompt" : "Return the extracted entity information as JSON object only: "
                                "{'Account_IDs': ['Acct_ID_1','Acct_ID_2','Acct_ID_3']}"
                                "{'FIs_to_Accts': { 'Entity1': ['Acct_ID_1'],'Entity2':['Acct_ID_2','Acct_ID_3]} }"
                                "{'Customers_to_Accts': { 'Entity2': ['Acct_ID_1'],'Entity4':['Acct_ID_2'],'Entity5':['Acct_ID_3']} }" 
    }
)

SAR_Agent (to Entity_Extraction_Agent):


This SAR is being filed to summarize suspicious cash deposits and wire transfer activity conducted by John Doe, account #12345678910.  John  Doe has been a bank customer since April 2000.  Mr. Doe is a college  student and employed part-time at Quickie Car Wash.  

Cash deposits to Mr. Doe’s personal checking account are structured to possibly circumvent federal reporting requirements.  The deposits are  followed by immediate wire transfers to Aussie Bank in Sydney, Australia to a single beneficiary, Jennifer Doe, account #981012345, with an address located in Australia.  Specifically the following activity has been 
observed: cash deposits (dates followed by amounts): 03/15/02  $9,950.00; 03/17/02 $9,700.00; 03/18/02 $10,000; total: $29,650.  Wire transfers out (dates followed by amounts): 03/16/02 $9,900.00, 03/18/02  $9,700.00, 03/19/02 $9,900.00.  

The volume and frequency of the deposits is not consistent with previous banking transaction

In [23]:
print(res.summary)

```json
{
  "Account_IDs": ["12345678910", "981012345"],
  "FIs_to_Accts": {
    "Aussie Bank": ["12345678910", "981012345"]
  },
  "Customers_to_Accts": {
    "John Doe": ["12345678910"],
    "Jennifer Doe": ["981012345"]
  }
}
```


# Sequential Chat with Narrative Extraction Agent

In [42]:
{"sar_narrative" :'''
This SAR is being filed to summarize suspicious cash deposits and wire transfer activity conducted by John Doe, account #12345678910.  John  Doe has been a bank customer since April 2000.  Mr. Doe is a college  student and employed part-time at Quickie Car Wash.  

Cash deposits to Mr. Doe’s personal checking account are structured to possibly circumvent federal reporting requirements.  The deposits are  followed by immediate wire transfers to Aussie Bank in Sydney, Australia to a single beneficiary, Jennifer Doe, account #981012345, with an address located in Australia.  Specifically the following activity has been 
observed: cash deposits (dates followed by amounts): 03/15/02  $9,950.00; 03/17/02 $9,700.00; 03/18/02 $10,000; total: $29,650.  Wire transfers out (dates followed by amounts): 03/16/02 $9,900.00, 03/18/02  $9,700.00, 03/19/02 $9,900.00.  

The volume and frequency of the deposits is not consistent with previous banking transactions conducted by Mr. Doe.  The amounts of currency do not appear consistent with the 
customer’s stated employment.  Also, the relationship between the customer and Jennifer Doe and the purpose for the wire activity is unknown. 
Therefore, due to the structured cash deposits by the customer on almost consecutive days into the account, and the immediate wire transfer of the funds out of the account to Jennifer Doe, Aussie bank, account  #981012345, Sydney Australia, this SAR is being filed.  Investigation is continuing. 
The bank’s financial intelligence unit in Big City, FL, maintains all records related to this SAR 
''',

"Customers_to_Accts": {
    "John Doe": [
      "12345678910"
    ],
    "Jennifer Doe": [
      "981012345"
    ]
  },

 "Account IDs": [
    "12345678910",
    "981012345"
  ]

}

{'sar_narrative': '\nThis SAR is being filed to summarize suspicious cash deposits and wire transfer activity conducted by John Doe, account #12345678910.  John  Doe has been a bank customer since April 2000.  Mr. Doe is a college  student and employed part-time at Quickie Car Wash.  \n\nCash deposits to Mr. Doe’s personal checking account are structured to possibly circumvent federal reporting requirements.  The deposits are  followed by immediate wire transfers to Aussie Bank in Sydney, Australia to a single beneficiary, Jennifer Doe, account #981012345, with an address located in Australia.  Specifically the following activity has been \nobserved: cash deposits (dates followed by amounts): 03/15/02  $9,950.00; 03/17/02 $9,700.00; 03/18/02 $10,000; total: $29,650.  Wire transfers out (dates followed by amounts): 03/16/02 $9,900.00, 03/18/02  $9,700.00, 03/19/02 $9,900.00.  \n\nThe volume and frequency of the deposits is not consistent with previous banking transactions conducted by M

In [135]:
narrative_extraction_agent = ConversableAgent(
    name="narrative_extraction_agent",
    system_message='''
    You are an AI Agent tasked with extracting or summarizing  parts of a narrative that describe activity conducted by certain accounts.

    You will have the following three pieces of information.

    1)  SAR narrative

    2)  The mapping between Individuals and Organizations referenced in the narrative to Account IDs which is given by the dictionary Customers_to_Accts

    3) List of Account IDs given by a list Account_IDs
    
    Using the SAR narrative shared by the user, Do the following:

    Step 1) Identify an Account ID from the list of Account IDs.Note that account IDs starting with the prefix "Dummy_" are placeholders for account IDs that have not been explicitly described in the narrative.

    Step 2)  From the dictionary Customers_to_Accts, Identify the  individual and Organization  who owns the account.

    Step 3) Extract or summarize the narrative which describes transactions involving this account.

    An example is given below, demarcated by the delimiter ----.

    ----

    1) SAR Narrative:
        John deposited $5000 in Cash into Acct #345723 at Bank of America. John sends $3000 to Jill's account at Chase. Jill deposited $3000 in Cash into her Acct at Chase Bank.
        John and Jill own a business Acme Inc that has a  Business account, Account #98765 at Bank of America. John sends $2000 from Acct #345723 to Account #98765. Jill sends $1000 from 
        her Acct at Chase Bank to Acct #98765.
    
    2) Customers_to_Accts = {"John":["345723"],"Jill":["Dummy_12345"],"Acme Inc":"98765"}

    3) Account_IDs = ["345723","98765","Dummy_12345"]

    
    Step 1) The first account ID  is 345723

    Step 2) This Account ID is owned by John.

    Step 3)Extract the part of the narrative that references the Account 345723

    {"345723": "John deposited $5000 in Cash into Acct #345723 at Bank of America. John sends $3000 to Jill's account at Chase.John sends $2000 from Acct #345723 to Account #98765" }

    
    Now repeat the process for the second account ID

    Step 1) The second  account ID  is 98765

    Step 2) This Account ID is owned by Acme, Inc.

    Step 3)Extract the part of the narrative that references the Account 98765

    {"98765": "John sends $2000 from Acct #345723 to Account #98765" }

    Now repeat the process for the third Account ID.

    Step 1) The third  account ID  is Dummy_12345. Given the acocunt ID starts with Dummy_ , there won't be direct references to this account ID in the narrative. 

    Step 2) This Account ID is owned by Jill. Given the Account ID is dummy, pay attention to transactions made by Jill,the owner of the dummy account.

    Step 3) Extract the part of the narrative that references the Account Dummy_12345

    {"Dummy_12345": "John sends $3000 to Jill's account at Chase. Jill deposited $3000 in Cash into her Acct at Chase Bank." }

    Consolidate narratives for all accounts.

     Narrative : {"345723": "John deposited $5000 in Cash into Acct #345723 at Bank of America. John sends $3000 to Jill's account at Chase.John sends $2000 from Acct #345723 to Account #98765",
      "98765": "John sends $2000 from Acct #345723 to Account #98765",
      "Dummy_12345": "John sends $3000 to Jill's account at Chase. Jill deposited $3000 in Cash into her Acct at Chase Bank." 
    }


    ----



    ''',
    llm_config=llm_config_large,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 09-03 19:43:39] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [68]:
chats = [
    {
        "sender": narrative_extraction_agent,
        "recipient": user_agent,
        "message": 
            "Hello, I'm here to help you extract relevant excerpts from the SAR narrative. "
            "Please provide the follwing informatiion"
            " { SAR Narrative: <SAR Narrative>,"
            "   Customers_to_Accts: <Dictionary mapping entites to accts> "
            "   Account_IDs: <List of Account IDs > }",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : '''Return the extracted  excerpts from the SAR narrative as JSON object only: 
                             {<Acct_1>: <Excerpt relevant to Acct_1>,
                              <Acct_2>: <Excerpt relevant to Acct_2>,
                              <Acct_3>: <Excerpt relevant to Acct_3> } 
                             '''
                             },                                                                      
        "max_turns": 1,
        "clear_history" : True
    }
]

In [69]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
narrative_extraction_agent (to user_agent):

Hello, I'm here to help you extract relevant excerpts from the SAR narrative. Please provide the follwing informatiion { SAR Narrative: <SAR Narrative>,   Customers_to_Accts: <Dictionary mapping entites to accts>    Account_IDs: <List of Account IDs > }

--------------------------------------------------------------------------------
user_agent (to narrative_extraction_agent):

{"sar_narrative" :''' This SAR is being filed to summarize suspicious cash deposits and wire transfer activity conducted by John Doe, account #12345678910.  John  Doe has been a bank customer since April 2000.  Mr. Doe is a college  student and employed part-time at Quickie Car Wash.    Cash deposits to Mr. Doe’s personal checking account are structured to possibly circumvent federal

In [70]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{
  "12345678910": "This SAR is being filed to summarize suspicious cash deposits and wire transfer activity conducted by John Doe, account #12345678910. John Doe has been a bank customer since April 2000. Mr. Doe is a college student and employed part-time at Quickie Car Wash. Cash deposits to Mr. Doe’s personal checking account are structured to possibly circumvent federal reporting requirements. The deposits are followed by immediate wire transfers to Aussie Bank in Sydney, Australia to a single beneficiary, Jennifer Doe, account #981012345, with an address located in Australia. Specifically the following activity has been observed: cash deposits (dates followed by amounts): 03/15/02 $9,950.00; 03/17/02 $9,700.00; 03/18/02 $10,000; total: $29,650. Wire transfers out (dates followed by amounts): 03/16/02 $9,900.00, 03/18/02 $9,700.00, 03/19/02 $9,900.00. The volume and frequency of the deposits is not consistent with previous banking transactions conducted by Mr. Doe. The amounts of 

# Sequential Chat with Entity Resolution

In [56]:


{
 "Account IDs": [
    "12345678910",
    "981012345"
  ],

  "Customers_to_Accts": {
    "John Doe": [
      "12345678910"
    ],
    "Jennifer Doe": [
      "981012345"
    ]
  },

  "FIs_to_Accts": {
    "Aussie Bank": [
      "981012345"
    ],
    "Dummy_Bank": [
      "12345678910"
    ]
  },

"sar_narrative" :'''
This SAR is being filed to summarize suspicious cash deposits and wire transfer activity conducted by John Doe, account #12345678910.  John  Doe has been a bank customer since April 2000.  Mr. Doe is a college  student and employed part-time at Quickie Car Wash.  

Cash deposits to Mr. Doe’s personal checking account are structured to possibly circumvent federal reporting requirements.  The deposits are  followed by immediate wire transfers to Aussie Bank in Sydney, Australia to a single beneficiary, Jennifer Doe, account #981012345, with an address located in Australia.  Specifically the following activity has been 
observed: cash deposits (dates followed by amounts): 03/15/02  $9,950.00; 03/17/02 $9,700.00; 03/18/02 $10,000; total: $29,650.  Wire transfers out (dates followed by amounts): 03/16/02 $9,900.00, 03/18/02  $9,700.00, 03/19/02 $9,900.00.  

The volume and frequency of the deposits is not consistent with previous banking transactions conducted by Mr. Doe.  The amounts of currency do not appear consistent with the 
customer’s stated employment.  Also, the relationship between the customer and Jennifer Doe and the purpose for the wire activity is unknown. 
Therefore, due to the structured cash deposits by the customer on almost consecutive days into the account, and the immediate wire transfer of the funds out of the account to Jennifer Doe, Aussie bank, account  #981012345, Sydney Australia, this SAR is being filed.  Investigation is continuing. 
The bank’s financial intelligence unit in Big City, FL, maintains all records related to this SAR 
'''

}

{'Account IDs': ['12345678910', '981012345'],
 'Customers_to_Accts': {'John Doe': ['12345678910'],
  'Jennifer Doe': ['981012345']},
 'FIs_to_Accts': {'Aussie Bank': ['981012345'], 'Dummy_Bank': ['12345678910']},
 'sar_narrative': '\nThis SAR is being filed to summarize suspicious cash deposits and wire transfer activity conducted by John Doe, account #12345678910.  John  Doe has been a bank customer since April 2000.  Mr. Doe is a college  student and employed part-time at Quickie Car Wash.  \n\nCash deposits to Mr. Doe’s personal checking account are structured to possibly circumvent federal reporting requirements.  The deposits are  followed by immediate wire transfers to Aussie Bank in Sydney, Australia to a single beneficiary, Jennifer Doe, account #981012345, with an address located in Australia.  Specifically the following activity has been \nobserved: cash deposits (dates followed by amounts): 03/15/02  $9,950.00; 03/17/02 $9,700.00; 03/18/02 $10,000; total: $29,650.  Wire tran

In [196]:
entity_resolution_agent = ConversableAgent(
    name="entity_resolution_agent",
    system_message='''
    You are an AI Agent tasked with assigning Customer IDs to entities described in a SAR and mapping Account IDs to these customer IDs.

    You will have the following four pieces of information.

    
    1) List of Account IDs given by the list Account_IDs. </n>
    2) The mapping between Individuals and Organizations to Account IDs  , given by the dictionary Customers_to_Accts. Note, account IDs starting with the prefix "Dummy_" </n>
    3) The mapping between Financial Institutions to Account IDs given by the dictionary FIs_to_Accts. If the Financial institution for certain accounts are not specified, a dummy financial institution
      "Dummy_Bank" will be used for them.</n>
    4) SAR narrative 
    
    Using this information, do the following:

    Step 1)  For each Financial institution in FIs_to_Accts, identify the accounts at that institution. Note, account IDs starting with the prefix "Dummy_" are placeholders for account IDs that
             have not been identified in the narrative.

    Step 2)  Identify the owners of the accounts from  Customers_to_Accts 

    Step 3)  If an Individual is a beneficial owner of an Organization and the Organizations's account, note this in a scratch pad.

    Step 4)  For each Financial institution, map each account ID to a Customer ID (E.g. CUST_001, CUST_002). If multiple accounts are owned by the same Individual or Organization, map them to the same Customer ID

    Return this final mapping between Customer IDs and Account IDs in the form of a JSON

    An example is given below, demarcated by the delimiter ----.

    ----

    1) Account_IDs = ["345723","98765","Dummy_12345"]
    
    2) Customers_to_Accts = {"John":["345723"],"Jill":["Dummy_12345"],"Acme Inc":"98765"}

    3) FIs_to_Accts =  {"Bank of America":["345723","98765"],"Chase Bank":["Dummy_12345"]}

    4) Narrative: </n>
    John deposited $5000 in Cash into Acct #345723 at Bank of America. John sends $3000 to Jill's account at Chase. Jill deposited $3000 in Cash into her Acct at Chase Bank.
    John and Jill own a business Acme Inc that has a  Business account, Account #98765 at Bank of America. John sends $2000 from Acct #345723 to Account #98765. Jill sends $1000 from 
    her Acct at Chase Bank to Acct #98765.

    
    Step 1) For each Financial institution in FIs_to_Accts, identify the accounts at that institution. 

    At Bank of America, there are two accounts #345723 and #98765
    At Chase Bank, there is one account #Dummy_12345

    Step 2) Identify the owners of the accounts from Customers_to_Accts 

    Account  #345723 is owned by John
    Account #Dummy_12345 is owned by Jill.
    Account #98765 is owned by Acme Inc

    Step 3)If an Individual is a beneficial owner of an Organization and the Organizations's account, note this in a scratch pad. If two individuals are join owners of an account, note this in a scratch pad.

    John and Jill are co-owners of the business Acme Inc and therefore its accounts.

    Step 4) For each Financial institution, map each account ID to a Customer ID (E.g. CUST_001, CUST_002). If multiple accounts are owned by the same Individual or Organizations, map them to the
    same Customer ID

    At Bank of America, the account #345723 is owned by John, the account 98765 is owned by Acme Inc. So these two account IDs can be mapped to two customer IDs. The Account Dummy_12345 
    is an account at Chase as per FIs_to_Accts and is owned by Jill as per Customers_to_Accts. This can be assigned to a different customer ID

    {"Bank of America": {"CUST_001": ["345723"], "CUST_002":["98765"]},
    "Chase Bank": {"CUST_003":["Dummy_12345"]}}
    
    Return this as a JSON.
    ----



    ''',
    llm_config=llm_config_mini,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 09-08 09:05:56] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [167]:
chats = [
    {
        "sender": entity_resolution_agent,
        "recipient": user_agent,
        "message": 
            "Hello, I'm here to help you assign Customer IDs for each account appropriately "
            "Please provide the follwing informatiion"
            " { "
            "   Account_IDs: <List of Account IDs > ,"
            "   Customers_to_Accts: <Dictionary mapping entites to accts> "
            "   FIs_to_Accts: <Dictionary mapping FIs to accts "
            "    Narrative: <The SAR Narrative > }",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : '''Return the extracted  excerpts from the SAR narrative as JSON object only: 
                                     'FI_to_Cust_to_Acct': {<Financial Institution 1>: { <CUST_ID_1>: [<ACCT_1>,<ACCT_2>], <CUST_ID_2>:[<ACCT_3>]},
                                     <Financial Institution 2>: { <CUST_ID_3>: [<ACCT_4>,<ACCT_5>], <CUST_ID_4>:[<ACCT_6>]} } 
                             '''
                             },                                                                      
        "max_turns": 1,
        "clear_history" : True
    }
]

In [168]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
entity_resolution_agent (to user_agent):

Hello, I'm here to help you assign Customer IDs for each account appropriately Please provide the follwing informatiion {    Account_IDs: <List of Account IDs > ,   Customers_to_Accts: <Dictionary mapping entites to accts>    FIs_to_Accts: <Dictionary mapping FIs to accts     Narrative: <The SAR Narrative > }

--------------------------------------------------------------------------------
user_agent (to entity_resolution_agent):

{  "Account IDs": [     "12345678910",     "981012345"   ],    "Customers_to_Accts": {     "John Doe": [       "12345678910"     ],     "Jennifer Doe": [       "981012345"     ]   },    "FIs_to_Accts": {     "Aussie Bank": [       "981012345"     ],     "Dummy_Bank": [       "12345678910"     ]   },  "sar_narrative" :''' This SAR is 

In [169]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

```json
{
  "FI_to_Cust_to_Acct": {
    "Aussie Bank": {
      "Jennifer Doe": [
        "981012345"
      ]
    },
    "Dummy_Bank": {
      "John Doe": [
        "12345678910"
      ]
    }
  }
}
```




# Sequential Chat with Entity Resolution - Updated with Account ID as Key

In [22]:
message = '''
  "Entities": {
        "Individuals": ["John Doe", "Jennifer Doe"],
        "Organizations": ["Quickie Car Wash"],
        "Financial_Institutions": ["Aussie Bank"]
    },
   "Account_IDs": ["12345678910", "981012345"],
   "FI_to_Acct": {
        "Dummy_Bank_1": ["12345678910"],
        "Aussie Bank": ["981012345"]
    },
   "Acct_to_Cust": {
        "12345678910": "John Doe",
        "981012345": "Jennifer Doe"
    },
    "SAR Narrative": "This SAR is being filed to summarize suspicious cash deposits and wire transfer activity conducted by John Doe, account #12345678910.  John  Doe has been a bank customer since April 2000.  Mr. Doe is a college  student and employed part-time at Quickie Car Wash.  

        Cash deposits to Mr. Doe’s personal checking account are structured to possibly circumvent federal reporting requirements.  The deposits are  followed by immediate wire transfers to Aussie Bank in Sydney, Australia to a single beneficiary, Jennifer Doe, account #981012345, with an address located in Australia.  Specifically the following activity has been 
        observed: cash deposits (dates followed by amounts): 03/15/02  $9,950.00; 03/17/02 $9,700.00; 03/18/02 $10,000; total: $29,650.  Wire transfers out (dates followed by amounts): 03/16/02 $9,900.00, 03/18/02  $9,700.00, 03/19/02 $9,900.00.  

        The volume and frequency of the deposits is not consistent with previous banking transactions conducted by Mr. Doe.  The amounts of currency do not appear consistent with the 
        customer’s stated employment.  Also, the relationship between the customer and Jennifer Doe and the purpose for the wire activity is unknown. 
        Therefore, due to the structured cash deposits by the customer on almost consecutive days into the account, and the immediate wire transfer of the funds out of the account to Jennifer Doe, Aussie bank, account  #981012345, Sydney Australia, this SAR is being filed.  Investigation is continuing. 
        The bank’s financial intelligence unit in Big City, FL, maintains all records related to this SAR"
'''

In [23]:
user_agent = ConversableAgent(
    name="user_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="NEVER"
)

In [24]:
entity_resolution_agent = ConversableAgent(
    name="entity_resolution_agent",
    system_message='''
    You are an AI Agent tasked with assigning Customer IDs to entities described in a SAR and mapping Account IDs to these customer IDs.

    You will have the following four pieces of information.

    
    1) List of Account IDs given by the list Account_IDs. </n>
    2) The mapping  from Account IDs to Entities (Individuals, Organizations and Financial Instituions), given by the dictionary Acct_to_Cust.  </n>
    3) The mapping from  Financial Instituions to Accts  given by the dictionary FI_to_Acct. If the Financial institution for certain accounts are not specified, a dummy financial institution
       e.g. "Dummy_Bank_1" will be used for them.</n>
    4) SAR narrative 
    
    Using this information, do the following:

    Step 1)  For each FI in FI_to_Acct, identify all the accounts held at that FI

    Step 2) For each such account at that FI, identify the owner of the accounts from Acct_to_Cust. 

    Step 3) Assign a unique customer ID (E.g. CUST_001, CUST_002) for each unique customer at the FI and map account ID to customer IDs
    If multiple accounts are owned by the same Individual or Organization, map them to the same Customer ID

    Return this final mapping between Customer IDs and Account IDs in the form of a JSON

    An example is given below, demarcated by the delimiter ----.

    ----

    1) Account_IDs = ["345723","98765","Dummy_12345"]
    
    2) Acct_to_Cust =  {"345723": "John,"Dummy_Acct_1" : "Jill","98765": "Acme Inc"}

    3) FI_to_Acct =  {"Bank of America":["345723","98765"],"Chase Bank":["Dummy_12345"]}

    4) Narrative: </n>
    John deposited $5000 in Cash into Acct #345723 at Bank of America. John sends $3000 to Jill's account at Chase. Jill deposited $3000 in Cash into her Acct at Chase Bank.
    John and Jill own a business Acme Inc that has a  Business account, Account #98765 at Bank of America. John sends $2000 from Acct #345723 to Account #98765. Jill sends $1000 from 
    her Acct at Chase Bank to Acct #98765.

    
    Step 1) For each Financial institution in FIs_to_Accts, identify the accounts at that institution. 

    At Bank of America, there are two accounts #345723 and #98765
    At Chase Bank, there is one account #Dummy_12345

    Step 2) Identify the owners of the accounts from Acct_to_Cust

    Account  #345723 is owned by John
    Account #Dummy_12345 is owned by Jill.
    Account #98765 is owned by Acme Inc

    Step 3) Assign a unique customer ID (E.g. CUST_001, CUST_002) for each unique customer at the FI and map account ID to customer IDs

    At Bank of America, there are two accounts as given in FIs_to_Acct,  the account #345723 is owned by John, the account #98765 is owned by Acme Inc as given in Acct_to_Cust. 
    So John can be assiged the customer ID CUST_001 and Acme Inc can be assiged the customer ID CUST_002.

    The Account #345723 owned by John can be mapped to CUST_001.
    The Account #98765 owned by Acme Inc can be mapped to CUST_002


    At Chase Bank , there is one account given in FI_to_Acct, the account Dummy_Acct_1 is owned by Jill. 
    So Jill can be assiged the customer ID CUST_003.

    The Account Dummy_Acct_1 owned by Jill can be mapped to CUST_003.

    Consolidate this information into a JSON

    {"Bank of America": {"CUST_001": ["345723"], "CUST_002":["98765"]},
    "Chase Bank": {"CUST_003":["Dummy_12345"]}}
    
    Return this as a JSON.
    ----



    ''',
    llm_config=llm_config_mini,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 11-17 11:12:49] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [26]:
chats = [
    {
        "sender": user_agent,
        "recipient": entity_resolution_agent,
        "message": message,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : '''Return the extracted information as a JSON object only: 
                                     'FI_to_Cust_to_Acct': {<Financial Institution 1>: { <CUST_ID_1>: [<ACCT_1>,<ACCT_2>], <CUST_ID_2>:[<ACCT_3>]},
                                     <Financial Institution 2>: { <CUST_ID_3>: [<ACCT_4>,<ACCT_5>], <CUST_ID_4>:[<ACCT_6>]} } 
                             '''                                                                    
        },
        "max_turns": 1,
        "clear_history" : False
    },
]

In [27]:
from autogen import initiate_chats
chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
user_agent (to entity_resolution_agent):


  "Entities": {
        "Individuals": ["John Doe", "Jennifer Doe"],
        "Organizations": ["Quickie Car Wash"],
        "Financial_Institutions": ["Aussie Bank"]
    },
   "Account_IDs": ["12345678910", "981012345"],
   "FI_to_Acct": {
        "Dummy_Bank_1": ["12345678910"],
        "Aussie Bank": ["981012345"]
    },
   "Acct_to_Cust": {
        "12345678910": "John Doe",
        "981012345": "Jennifer Doe"
    },
    "SAR Narrative": "This SAR is being filed to summarize suspicious cash deposits and wire transfer activity conducted by John Doe, account #12345678910.  John  Doe has been a bank customer since April 2000.  Mr. Doe is a college  student and employed part-time at Quickie Car Wash.  

        Cash deposits to Mr. Doe’s personal checking acco

In [28]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

```json
{
    "FI_to_Cust_to_Acct": {
        "Dummy_Bank_1": {
            "CUST_001": ["12345678910"]
        },
        "Aussie Bank": {
            "CUST_002": ["981012345"]
        }
    }
}
```




# Sequential Chat with Trxn Generation

In [64]:
{
 "Bank_to_Cust_to_Acct": {
  "Aussie Bank": {
    "Jennifer Doe": [
      "981012345"
    ]
  },
  "Dummy_Bank": {
    "John Doe": [
      "12345678910"
    ]
  }
},

  "Customers_to_Accts": {
    "John Doe": [
      "12345678910"
    ],
    "Jennifer Doe": [
      "981012345"
    ]
  },

  "FIs_to_Accts": {
    "Aussie Bank": [
      "981012345"
    ],
    "Dummy_Bank": [
      "12345678910"
    ]
  },

"Narrative" : {
    "12345678910": "John Doe has been a bank customer since April 2000. Cash deposits to Mr. Doe’s personal checking account are structured to possibly circumvent federal reporting requirements. The volume and frequency of the deposits is not consistent with previous banking transactions conducted by Mr. Doe. The amounts of currency do not appear consistent with the customer’s stated employment. Therefore, due to the structured cash deposits by the customer on almost consecutive days into the account, and the immediate wire transfer of the funds out of the account to Jennifer Doe, this SAR is being filed."
}


}

{'Bank_to_Cust_to_Acct': {'Aussie Bank': {'Jennifer Doe': ['981012345']},
  'Dummy_Bank': {'John Doe': ['12345678910']}},
 'Customers_to_Accts': {'John Doe': ['12345678910'],
  'Jennifer Doe': ['981012345']},
 'FIs_to_Accts': {'Aussie Bank': ['981012345'], 'Dummy_Bank': ['12345678910']},
 'Narrative': {'12345678910': 'John Doe has been a bank customer since April 2000. Cash deposits to Mr. Doe’s personal checking account are structured to possibly circumvent federal reporting requirements. The volume and frequency of the deposits is not consistent with previous banking transactions conducted by Mr. Doe. The amounts of currency do not appear consistent with the customer’s stated employment. Therefore, due to the structured cash deposits by the customer on almost consecutive days into the account, and the immediate wire transfer of the funds out of the account to Jennifer Doe, this SAR is being filed.'}}

## <Update prompt to specify that for cash transactions. Originator and Beneficary can be the same>

In [193]:
trxn_generation_agent = ConversableAgent(
    name="trxn_generation_agent",
    system_message='''
    You are an AI Agent tasked with synthesizing transactions from a narrative you are provided. Your strength is synthesizing a small number of transactions, especially when 
    transaction amounts and dates are explicitly listed.

    You will have the following four pieces of information.

    
    1) A JSON called Narrative with Account IDs as key  and  narratives potentially describing transactions made by that account as the values. </n>
    2) The mapping between Individuals and Organizations referenced in the narrative to Account IDs, given by the dictionary Customers_to_Accts. Note, account IDs starting with the prefix "Dummy_" 
    are placeholders for account IDs that have not been identified in the narrative. </n>
    3) The mapping between Financial Institutions to Account IDs given by the dictionary FIs_to_Accts. If the Financial institution for certain accounts are not specified in the narrative,
      a dummy financial institution "Dummy_Bank" will be used for them.</n>
    4) A mapping between Financial institution, Customer IDs and Accounts owned by those Customer IDs given by the dictionary FI_to_Cust_to_Acct
    
    For the narrative corresponding  to each account ID, do the following:

    Step 1) Identify the number of transactions described in the narrative. Note this in a scratch pad for verification later.

    
    Step 2)For each transaction involving the account ID,  

    a)  Identify the transaction channel or product used. This could be Wire, Cash, Checks or something similar. If this is missing, you can make a reasonable assumption.

    b) Identify the Originator Account ID, the Customer ID corresponding to the account, the Originating_Name which is the Individual Or Organization originating
    the transaction.

    c)  Identify the Beneficiary Account ID, the Customer ID corresponding to the account, the beneficiary_Name which is the Individual Or Organization which is the 
    Beneficiary of the transaction. Note that for cash transactions, if only information on the originator or benficary are available, you can assume the originator and beneficiary are the same.

    d) Identify the date and amount of the transaction. If this is missing, you can make reasonable assumptions.

    e) If this is a Cash or Check or similar transaction, determine the Branch or ATM Location where the transaction was conducted. If this is not specified, indicate it as "Missing"

    Ensure the following are extracted.

    -Originator_Name
    -Originator_Account_ID
    -Originator_Customer_ID
    -Beneficiary_Name
    -Beneficiary_Account_ID
    -Beneficiary_Customer_ID
    -Trxn_Channel
    -Trxn_Date in yyyy-mm-dd format
    -Trxn_Amount
    -Branch or ATM Location

    If any information is missing and can't be reasonably assumed, use an empty string "" to indicate it is missing.

    Step 3) Ensure the number of transactions extracted in Step 2 are the same as that noted in Step 1. if this is not the case, go back to Step 2.

    Step 4) Return the transactions as a JSON file with the key being a Trxn ID and value being the transaction attributes

    An example is given below, demarcated by the delimiter ----..

    ----

    Narrative = {"345723": "John deposited $5000 in Cash into Acct #345723 at the Main Road, NY Branch of Bank of America on Jan 4, 2024.
                  John sends $3000 to Acme Inc's account at Bank of America by Wire on Jan 6, 2024 . John  wrote a check to Jill from Acct #345723 on Jan 8,2024 for $1,000"}

                  Customers_to_Accts = {"John":["345723"],"Jill":["Dummy_12345"],"Acme Inc":"98765"}
                  FIs_to_Accts = {"Bank of America":["345723","98765"],"Chase Bank":["Dummy_12345"]}
                  FI_to_Cust_to_Acct = {"Bank of America": {"CUST_001": ["345723"], "CUST_002":["98765"]}, "Chase Bank": {"CUST_003":["Dummy_12345"]}}

    
    Step 1) Identify the number of transactions described in the narrative. Note this is a scratch pad for verification later.
      There are three trxns described in the narrative. A $5000 deposit, A $3000 transfer and a $3000 transfer.

   Step 2) For each transaction, identify the required attributes.

   For the first transaction, the Beneficiary_Name is John as the money is being deposited into John's account. The Beneficiary_Account_ID is #345723. From Bank_to_Cust_to_Acct, 
   The Beneficiary Customer ID is CUST_001.

   The Originator_Name is John, but this is a cash deposit , so there is no Account ID specified from which the cash originates.
   Given this is a cash deposit and information on the Originator is unavailable, it can be assumed to be the same the Beneficary Information.

   The Trxn Channel is Cash as it is a cash deposit.
   The Trxn Date is  2024-01-04.
   The Trxn Amount is $5,000.
   The Branch was Main Road , NY

   For the second transaction, the Originator_Name is John. Although no account ID is specified, it is a wire transaction following a cash transaction, 
   so it has to come from John's account #345723.The Originator_Account_ID is #345723. The Originator Customer ID is CUST_001 as per Bank_to_Cust_to_Acct.

   The Beneficiary_Name is Acme Inc. The Beneficiary_Account_ID is #98765 as this account belongs to Acme Inc at Bank of America.
   From Bank_to_Cust_to_Acct, The Beneficiary Customer ID is CUST_002.


    The Trxn Channel is Wire.
    The Trxn Date is  2024-01-06.
    The Trxn Amount is $3,000.
    The Branch is Missing as it is a wire transaction.

    For the third transaction, the Originator_Name is John. The Originator_Account_ID is 345723 . The Originator Customer ID is CUST_001 as per Bank_to_Cust_to_Acct.

    The Beneficiary_Name is Jill. The Beneficiary_Account_ID must be #Dummy_12345 as the information provided indicated that this is Jill's account at Chase Bank.
    From Bank_to_Cust_to_Acct, The Beneficiary Customer ID is CUST_003.


    The Trxn Channel is Check
    The Trxn Date is  2024-01-08.
    The Trxn Amount is $1,000.
    The Branch is Missing as it is a Check.

    Step 3) Ensure the number of transactions extracted in Step 2 are the same as that noted in Step 1. if this is not the case, go back to Step 2.

    Three transactions have been identified, which is the same as the number of transaction in scratch pad from Step 1)

    Step 4) Return the transactions as a JSON file with the key being a Trxn ID and value being the transaction attributes

    Now the transactions can be combined into a JSON file with a key being a Trxn ID staring from 1.

    Trxns = {"345723": {1 : {"Originator_Name": "John", "Originator_Account_ID": "", "Originator_Customer_ID":"","Beneficiary_Name": "John","Beneficiary_Account_ID":"345723",
              "Beneficiary_Customer_ID":"CUST_001" ,"Trxn_Channel":"Cash", "Trxn_Date":"2024-01-04", "Trxn_Amount":5000, "Branch or ATM Location": "Main Road,NY"
            },
              2: {"Originator_Name": "John", "Originator_Account_ID": "345723", "Originator_Customer_ID":"CUST_001","Beneficiary_Name": "Acme,Inc","Beneficiary_Account_ID":"98765",
              "Beneficiary_Customer_ID":"CUST_002" ,"Trxn_Channel":"Wire", "Trxn_Date":"2024-01-06", "Trxn_Amount":3000, "Branch or ATM Location": ""
            },
            3: {"Originator_Name": "John", "Originator_Account_ID": "345723", "Originator_Customer_ID":"CUST_001","Beneficiary_Name": "Jill","Beneficiary_Account_ID":"Dummy_12345",
              "Beneficiary_Customer_ID":"CUST_003" ,"Trxn_Channel":"Check", "Trxn_Date":"2024-01-08", "Trxn_Amount":1000, "Branch or ATM Location": ""

        }
    }

    ----



    ''',
    llm_config=llm_config_mini,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 10-08 20:17:38] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [70]:
chats = [
    {
        "sender": trxn_generation_agent,
        "recipient": user_agent,
        "message": 
            "Hello, I'm here to help  synthesize trxns from a narrative "
            "Please provide the follwing informatiion"
            " { "
            "   Bank_to_Cust_to_Acct: <Mapping between Financial institutions to Customers , and Customers to Accts > ,"
            "   Customers_to_Accts: <Dictionary mapping entites to accts> "
            "   FIs_to_Accts: <Dictionary mapping FIs to accts "
            "   Narrative: <The  Narrative to synthesize trxns from > }",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : '''Return the  synthesized trxns from the SAR narrative as JSON object only: 
                             {<Trxn_ID> : {"Originator_Name": <Originator_Name>, "Originator_Account_ID": <Originator_Account_ID>, 
                                          "Originator_Customer_ID": <Originator_Customer_ID>,"Beneficiary_Name": <Beneficiary_Name>,
                                           "Beneficiary_Account_ID":<Beneficiary_Account_ID>, "Beneficiary_Customer_ID": <Beneficiary_Customer_ID> ,
                                           "Trxn_Channel": <Trxn_Channel>, "Trxn_Date":<Trxn_Date>, "Trxn_Amount":<Trxn_Amount>, "Branch_or_ATM_Location": <Branch_or_ATM_Location>
                                    } 
                             '''
                             },                                                                      
        "max_turns": 1,
        "clear_history" : True
    }
]

In [64]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
trxn_generation_agent (to user_agent):

Hello, I'm here to help  synthesize trxns from a narrative Please provide the follwing informatiion {    Bank_to_Cust_to_Acct: <Mapping between Financial institutions to Customers , and Customers to Accts > ,   Customers_to_Accts: <Dictionary mapping entites to accts>    FIs_to_Accts: <Dictionary mapping FIs to accts    Narrative: <The  Narrative to synthesize trxns from > }

--------------------------------------------------------------------------------
user_agent (to trxn_generation_agent):

{  "Bank_to_Cust_to_Acct": {   "Aussie Bank": {     "Jennifer Doe": [       "981012345"     ]   },   "Dummy_Bank": {     "John Doe": [       "12345678910"     ]   } },    "Customers_to_Accts": {     "John Doe": [       "12345678910"     ],     "Jennifer Doe": [       "981

In [65]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

```json
{
    "TRXN001": {
        "Originator_Name": "John Doe",
        "Originator_Account_ID": "12345678910",
        "Originator_Customer_ID": "John Doe",
        "Beneficiary_Name": "Jennifer Doe",
        "Beneficiary_Account_ID": "981012345",
        "Beneficiary_Customer_ID": "Jennifer Doe",
        "Trxn_Channel": "Wire Transfer",
        "Trxn_Date": "YYYY-MM-DD",
        "Trxn_Amount": "Varies",
        "Branch or ATM Location": "N/A"
    }
}
``` 

(Note: Replace "YYYY-MM-DD" and "Varies" with actual transaction date and amount if known from the narrative.)




# Sequential Chat for  Full Workflow

In [129]:
sar_agent = ConversableAgent(
    name="SAR_Agent",
    system_message="You are assigned a SAR from which to extract transactions. You do this by sharing this SAR with other assistant agents who "
                   "work on the SAR and extract useful information.      ",
    llm_config=llm_config_mini,
    human_input_mode="NEVER",
)

[autogen.oai.client: 09-02 12:07:54] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [197]:

sar1 = '''
Investigation case number: A5678910. The customer, a grocery store and  its owner, are suspected of intentionally structuring cash deposits to  circumvent federal reporting requirements.  The customer is also  engaged in activity indicative of an informal value transfer operation: deposits of bulk cash, third party out of state personal checks and money 
orders, and engaging in aggregate wire transfers to Dubai, UAE.  The type and volume of activity observed is non-commensurate with the customer’s expected business volume and deviates from the normal volume of similar types of businesses located in the same area as the customer.  Investigative activities are continuing. Our bank has elected to 
directly contact law enforcement concerning this matter along with filing this SAR. 

John Doe opened a personal checking account, #12345-6789, in March of 1994.  Doe indicated that he was born in Yemen, presented a Virginia driver's license as identification, and claimed he was the self-employed  owner of a grocery store identified as Acme, Inc.  A business checking  account, #23456-7891, was opened in January of 1998 for Acme, Inc.  
   
Between January 17, 2003, and March 21, 2003, John Doe was the  originator of nine wires totaling $225,000.  The wire transfers were always  conducted at the end of each week in the amount of $25,000.  All of the  wires were remitted to the Bank of Anan in Dubai, UAE, to benefit Kulkutta Building Supply Company, account #3489728.   

Reviews covering the period between January 2 and March 17, 2003, revealed that 13 deposits (consisting of cash, checks, money orders) totaling approximately $50,000 posted to the personal account.  Individual amounts ranged between $1,500 and $9,500 and occurred on consecutive business days in several instances.  A number of third-party out of state checks and money orders were also deposited into the account.

A review of deposit activity on the Acme, Inc. account covering the same  period revealed 33 deposits (consisting of cash, checks, money orders) totaling approximately $275,000.  Individual amounts ranged between  $4,446 and $9,729; however 22 of 33 deposits ranged between $9,150  and $9,980.  It was further noted that in nine of 13 instances in which cash deposits were made to both accounts on the same day, the  combined deposits of cash exceeded $10,000. The bank filed currency  transaction reports to the IRS for all aggregate daily transactions  exceeding $10,000. 

A search of the world wide web identified a website for Acme, Inc., which  identified the company as a grocery store that provides remittance  services to countries in the Middle East that includes Iran (an OFAC blocked country).  Contact with the Virginia State Department of Banking  indicates Acme, Inc. is not a licensed money wire transfer business.  The 
bank will close this account because of the suspect nature of the transactions being conducted by John Doe. '''

message = f'''
 Please analyze the following supicious activity report:
 {sar1}

'''

In [198]:
chat_results = sar_agent.initiate_chats(
      [
        {
            "recipient": entity_extraction_agent,
            "message": message,
            "max_turns": 1,
            "summary_method": "reflection_with_llm",
            "summary_args": {
                "summary_prompt" : "Return the extracted entity information s JSON object only: "
                                "{'Account IDs': ['Acct_ID_1','Acct_ID_2','Acct_ID_3']}"
                                "{'FIs_to_Accts': { 'Entity4': ['Acct_ID_1'],'Entity5':['Acct_ID_2','Acct_ID_3]} }"
                                "{'Customers_to_Accts': { 'Entity1': ['Acct_ID_1'],'Entity2':['Acct_ID_2'],'Entity3':['Acct_ID_3']} }                                                                      ",
            },
        },
        {
            "recipient": entity_resolution_agent,
            "message": message,
            "max_turns": 1,
            "summary_method": "reflection_with_llm",
            "summary_args": {
                "summary_prompt" : '''Return the resolved entites in the SAR narrative as JSON object only: 
                                {'FI_to_Cust_to_Acct':{<Financial Institution 1>: { <CUST_ID_1>: [<ACCT_1>,<ACCT_2>], <CUST_ID_2>:[<ACCT_3>]},
                                                      <Financial Institution 2>: { <CUST_ID_3>: [<ACCT_4>,<ACCT_5>], <CUST_ID_4>:[<ACCT_6>]} } 
                                                      }
                                '''
                                },
        },
        {
            "recipient": narrative_extraction_agent,
            "message": message,
            "max_turns": 1,
            "summary_method": "reflection_with_llm",
            "summary_args": {
                "summary_prompt" : '''Return the extracted  excerpts from the SAR narrative as JSON object only: 
                                    'Narrative': {<Acct_1>: <Excerpt relevant to Acct_1>,
                                                  <Acct_2>: <Excerpt relevant to Acct_2>,
                                                  <Acct_3>: <Excerpt relevant to Acct_3> } 
                                   '''
                                },
        },

         {
            "recipient": trxn_generation_agent,
            "message": message,
            "max_turns": 1,
            "summary_method": "reflection_with_llm",
            "summary_args": {
                "summary_prompt" : '''Return the  synthesized trxns from the SAR narrative as JSON object only: 
                                        { <Acct_ID> : {<Trxn_ID> : {"Originator_Name": <Originator_Name>, "Originator_Account_ID": <Originator_Account_ID>, "Originator_Customer_ID": <Originator_Customer_ID>,
                                                       "Beneficiary_Name": <Beneficiary_Name>,"Beneficiary_Account_ID":<Beneficiary_Account_ID>, "Beneficiary_Customer_ID": <Beneficiary_Customer_ID> ,
                                                       "Trxn_Channel": <Trxn_Channel>, "Trxn_Date":<Trxn_Date>, "Trxn_Amount":<Trxn_Amount>, 
                                                       "Branch or ATM Location": <Branch or ATM Location> }
                                    } 
                             '''
                                },
        }



       
    ]


)


********************************************************************************
Starting a new chat....

********************************************************************************
SAR_Agent (to Entity Extraction Agent):


 Please analyze the following supicious activity report:
 
Investigation case number: A5678910. The customer, a grocery store and  its owner, are suspected of intentionally structuring cash deposits to  circumvent federal reporting requirements.  The customer is also  engaged in activity indicative of an informal value transfer operation: deposits of bulk cash, third party out of state personal checks and money 
orders, and engaging in aggregate wire transfers to Dubai, UAE.  The type and volume of activity observed is non-commensurate with the customer’s expected business volume and deviates from the normal volume of similar types of businesses located in the same area as the customer.  Investigative activities are continuing. Our bank has elected to 
directl

In [199]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

```json
{
    "Account IDs": ["12345-6789", "23456-7891", "3489728"],
    "FIs_to_Accts": {
        "Dummy_Bank": ["12345-6789", "23456-7891"],
        "Bank of Anan": ["3489728"]
    },
    "Customers_to_Accts": {
        "John Doe": ["12345-6789"],
        "Acme, Inc.": ["23456-7891"],
        "Kulkutta Building Supply Company": ["3489728"]
    }
}
```


```json
{
    "FI_to_Cust_to_Acct": {
        "Dummy_Bank": {
            "CUST_001": ["12345-6789", "23456-7891"]
        },
        "Bank of Anan": {
            "CUST_002": ["3489728"]
        }
    }
}
```


```json
{
  "Narrative": {
    "12345-6789": "John Doe opened a personal checking account, #12345-6789, in March of 1994. Reviews covering the period between January 2 and March 17, 2003, revealed that 13 deposits (consisting of cash, checks, money orders) totaling approximately $50,000 posted to the personal account. Individual amounts ranged between $1,500 and $9,500 and occurred on consecutive business days in several inst

In [200]:
chat_results[3].chat_history

[{'content': '\n Please analyze the following supicious activity report:\n \nInvestigation case number: A5678910. The customer, a grocery store and  its owner, are suspected of intentionally structuring cash deposits to  circumvent federal reporting requirements.  The customer is also  engaged in activity indicative of an informal value transfer operation: deposits of bulk cash, third party out of state personal checks and money \norders, and engaging in aggregate wire transfers to Dubai, UAE.  The type and volume of activity observed is non-commensurate with the customer’s expected business volume and deviates from the normal volume of similar types of businesses located in the same area as the customer.  Investigative activities are continuing. Our bank has elected to \ndirectly contact law enforcement concerning this matter along with filing this SAR. \n\nJohn Doe opened a personal checking account, #12345-6789, in March of 1994.  Doe indicated that he was born in Yemen, presented a

In [201]:
chat_results[3].summary

'```json\n{\n  "12345-6789": {\n    "Trxn_1": {\n      "Originator_Name": "John Doe",\n      "Originator_Account_ID": "12345-6789",\n      "Originator_Customer_ID": "CUST_001",\n      "Beneficiary_Name": "John Doe",\n      "Beneficiary_Account_ID": "12345-6789",\n      "Beneficiary_Customer_ID": "CUST_001",\n      "Trxn_Channel": "Cash",\n      "Trxn_Date": "2003-01-02",\n      "Trxn_Amount": 1500,\n      "Branch or ATM Location": "Missing"\n    },\n    "Trxn_2": {\n      "Originator_Name": "John Doe",\n      "Originator_Account_ID": "12345-6789",\n      "Originator_Customer_ID": "CUST_001",\n      "Beneficiary_Name": "John Doe",\n      "Beneficiary_Account_ID": "12345-6789",\n      "Beneficiary_Customer_ID": "CUST_001",\n      "Trxn_Channel": "Cash",\n      "Trxn_Date": "2003-01-03",\n      "Trxn_Amount": 3000,\n      "Branch or ATM Location": "Missing"\n    },\n    "Trxn_3": {\n      "Originator_Name": "John Doe",\n      "Originator_Account_ID": "12345-6789",\n      "Originator_Custo

In [208]:
cleaned_trxns = chat_results[3].summary.split('```',2)[1]
trxns = cleaned_trxns.strip('```json').strip('```')

In [209]:
print(trxns)


{
  "12345-6789": {
    "Trxn_1": {
      "Originator_Name": "John Doe",
      "Originator_Account_ID": "12345-6789",
      "Originator_Customer_ID": "CUST_001",
      "Beneficiary_Name": "John Doe",
      "Beneficiary_Account_ID": "12345-6789",
      "Beneficiary_Customer_ID": "CUST_001",
      "Trxn_Channel": "Cash",
      "Trxn_Date": "2003-01-02",
      "Trxn_Amount": 1500,
      "Branch or ATM Location": "Missing"
    },
    "Trxn_2": {
      "Originator_Name": "John Doe",
      "Originator_Account_ID": "12345-6789",
      "Originator_Customer_ID": "CUST_001",
      "Beneficiary_Name": "John Doe",
      "Beneficiary_Account_ID": "12345-6789",
      "Beneficiary_Customer_ID": "CUST_001",
      "Trxn_Channel": "Cash",
      "Trxn_Date": "2003-01-03",
      "Trxn_Amount": 3000,
      "Branch or ATM Location": "Missing"
    },
    "Trxn_3": {
      "Originator_Name": "John Doe",
      "Originator_Account_ID": "12345-6789",
      "Originator_Customer_ID": "CUST_001",
      "Beneficiar

In [210]:
import json

result_dict = json.loads(trxns)

In [212]:
result_dict.keys()

dict_keys(['12345-6789', '23456-7891', '3489728'])

In [213]:
pd.DataFrame.from_dict(result_dict['12345-6789'],orient='index')

,Originator_Name,Originator_Account_ID,Originator_Customer_ID,Beneficiary_Name,Beneficiary_Account_ID,Beneficiary_Customer_ID,Trxn_Channel,Trxn_Date,Trxn_Amount,Branch or ATM Location
Trxn_1,John Doe,12345-6789,CUST_001,John Doe,12345-6789,CUST_001,Cash,2003-01-02,1500,Missing
Trxn_2,John Doe,12345-6789,CUST_001,John Doe,12345-6789,CUST_001,Cash,2003-01-03,3000,Missing
Trxn_3,John Doe,12345-6789,CUST_001,John Doe,12345-6789,CUST_001,Cash,2003-01-04,4500,Missing
Trxn_4,John Doe,12345-6789,CUST_001,John Doe,12345-6789,CUST_001,Cash,2003-01-05,2000,Missing
Trxn_5,John Doe,12345-6789,CUST_001,John Doe,12345-6789,CUST_001,Cash,2003-01-06,2500,Missing
Trxn_6,John Doe,12345-6789,CUST_001,John Doe,12345-6789,CUST_001,Cash,2003-01-07,7500,Missing
Trxn_7,John Doe,12345-6789,CUST_001,John Doe,12345-6789,CUST_001,Cash,2003-01-08,4000,Missing
Trxn_8,John Doe,12345-6789,CUST_001,John Doe,12345-6789,CUST_001,Cash,2003-01-09,5000,Missing
Trxn_9,John Doe,12345-6789,CUST_001,John Doe,12345-6789,CUST_001,Cash,2003-01-10,9500,Missing
Trxn_10,John Doe,12345-6789,CUST_001,John Doe,12345-6789,CUST_001,Cash,2003-01-11,6000,Missing


In [214]:
pd.DataFrame.from_dict(result_dict['23456-7891'],orient='index')

,Originator_Name,Originator_Account_ID,Originator_Customer_ID,Beneficiary_Name,Beneficiary_Account_ID,Beneficiary_Customer_ID,Trxn_Channel,Trxn_Date,Trxn_Amount,Branch or ATM Location
Trxn_1,John Doe,23456-7891,CUST_002,"Acme, Inc.",23456-7891,CUST_002,Cash,2003-01-02,9000,Missing
Trxn_2,John Doe,23456-7891,CUST_002,"Acme, Inc.",23456-7891,CUST_002,Cash,2003-01-03,9500,Missing
Trxn_3,John Doe,23456-7891,CUST_002,"Acme, Inc.",23456-7891,CUST_002,Cash,2003-01-04,4500,Missing
Trxn_4,John Doe,23456-7891,CUST_002,"Acme, Inc.",23456-7891,CUST_002,Cash,2003-01-05,8400,Missing
Trxn_5,John Doe,23456-7891,CUST_002,"Acme, Inc.",23456-7891,CUST_002,Cash,2003-01-06,7500,Missing
Trxn_6,John Doe,23456-7891,CUST_002,"Acme, Inc.",23456-7891,CUST_002,Cash,2003-01-07,8000,Missing
Trxn_7,John Doe,23456-7891,CUST_002,"Acme, Inc.",23456-7891,CUST_002,Cash,2003-01-08,9600,Missing
Trxn_8,John Doe,23456-7891,CUST_002,"Acme, Inc.",23456-7891,CUST_002,Cash,2003-01-09,6700,Missing
Trxn_9,John Doe,23456-7891,CUST_002,"Acme, Inc.",23456-7891,CUST_002,Cash,2003-01-10,5500,Missing
Trxn_10,John Doe,23456-7891,CUST_002,"Acme, Inc.",23456-7891,CUST_002,Cash,2003-01-11,7200,Missing


In [215]:
pd.DataFrame.from_dict(result_dict['3489728'],orient='index')

,Originator_Name,Originator_Account_ID,Originator_Customer_ID,Beneficiary_Name,Beneficiary_Account_ID,Beneficiary_Customer_ID,Trxn_Channel,Trxn_Date,Trxn_Amount,Branch or ATM Location
Trxn_1,John Doe,12345-6789,CUST_001,Kulkutta Building Supply Company,3489728,CUST_002,Wire,2003-01-19,25000,Missing
Trxn_2,John Doe,12345-6789,CUST_001,Kulkutta Building Supply Company,3489728,CUST_002,Wire,2003-01-26,25000,Missing
Trxn_3,John Doe,12345-6789,CUST_001,Kulkutta Building Supply Company,3489728,CUST_002,Wire,2003-02-02,25000,Missing
Trxn_4,John Doe,12345-6789,CUST_001,Kulkutta Building Supply Company,3489728,CUST_002,Wire,2003-02-09,25000,Missing
Trxn_5,John Doe,12345-6789,CUST_001,Kulkutta Building Supply Company,3489728,CUST_002,Wire,2003-02-16,25000,Missing
Trxn_6,John Doe,12345-6789,CUST_001,Kulkutta Building Supply Company,3489728,CUST_002,Wire,2003-02-23,25000,Missing
Trxn_7,John Doe,12345-6789,CUST_001,Kulkutta Building Supply Company,3489728,CUST_002,Wire,2003-03-02,25000,Missing
Trxn_8,John Doe,12345-6789,CUST_001,Kulkutta Building Supply Company,3489728,CUST_002,Wire,2003-03-09,25000,Missing
Trxn_9,John Doe,12345-6789,CUST_001,Kulkutta Building Supply Company,3489728,CUST_002,Wire,2003-03-16,25000,Missing


In [216]:
chat_result.cost

{'usage_including_cached_inference': {'total_cost': 0,
  'gpt-4o-mini-2024-07-18': {'cost': 0,
   'prompt_tokens': 14682,
   'completion_tokens': 11443,
   'total_tokens': 26125}},
 'usage_excluding_cached_inference': {'total_cost': 0,
  'gpt-4o-mini-2024-07-18': {'cost': 0,
   'prompt_tokens': 14682,
   'completion_tokens': 11443,
   'total_tokens': 26125}}}

In [145]:
from dataclasses import asdict
chat_result_dict = asdict(chat_result)
chat_result_dict.keys()

dict_keys(['chat_id', 'chat_history', 'summary', 'cost', 'human_input'])

# Try Tool use with Local Executor

In [217]:
import tempfile

In [218]:
from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

In [223]:
# Create a temporary directory to store the code files.
temp_dir = tempfile.TemporaryDirectory()

In [224]:
# Create a local command line code executor.
executor = LocalCommandLineCodeExecutor(
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

In [225]:
code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=False,
    code_execution_config={"executor":executor},
    human_input_mode="ALWAYS"
)

In [226]:
message_with_code_block = """This is a message with code block.
The code block is below:
```python
import numpy as np
import matplotlib.pyplot as plt
x = np.random.randint(0, 100, 100)
y = np.random.randint(0, 100, 100)
plt.scatter(x, y)
plt.savefig('scatter.png')
print('Scatter plot saved to scatter.png')
```
This is the end of the message.
"""

In [227]:
# Generate a reply for the given code.
reply = code_executor_agent.generate_reply(messages=[{"role": "user", "content": message_with_code_block}])
print(reply)


>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
exitcode: 0 (execution succeeded)
Code output: Scatter plot saved to scatter.png



In [228]:
import os

print(os.listdir(temp_dir.name))

['scatter.png', 'tmp_code_e24bf32d4a21990fb9e4b5eb889ebe5a.py']


### Simple Calculator Tool

In [152]:
import os

from autogen import ConversableAgent

# Let's first define the assistant agent that suggests tool calls.
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with simple calculations. "
    "Return 'TERMINATE' when the task is done.",
    llm_config= llm_config_mini,
)

[autogen.oai.client: 10-08 19:18:42] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [153]:
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

In [154]:
from typing import Annotated, Literal

Operator = Literal["+", "-", "*", "/"]

@user_proxy.register_for_execution(name="calculator")
@assistant.register_for_llm(name="calculator",description="A simple calculator")
def calculator(a: int, b: int, operator: Annotated[Operator, "operator for calculations"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")

[autogen.oai.client: 10-08 19:18:44] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [155]:
chat_result = user_proxy.initiate_chat(assistant, message = "What is ((2+5)^3 -34) ?")

User (to Assistant):

What is ((2+5)^3 -34) ?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_RnTtYObjAErRKs1BVPJMIgCf): calculator *****
Arguments: 
{"a":2,"b":5,"operator":"+"}
***************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculator...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_RnTtYObjAErRKs1BVPJMIgCf) *****
7
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_I8ym4tDSsOPHOPzhHhRtx7u5): calculator *****
Arguments: 
{"a":7,"b":3,"operator":"^"}
***************************************************************

### Simple Adder Tool using Group Chat

In [32]:

import autogen
import openai

print(autogen.__version__,openai.__version__)

0.3.0 1.70.0


In [33]:
user_agent = autogen.ConversableAgent(
    name="user_agent",
    llm_config=False,
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": False,
    },
    human_input_mode="NEVER"
)

In [34]:

small_multiplier_agent = autogen.ConversableAgent(
    name="Small_Multiplier_Agent",
    system_message='''
        You are an assistant who helps with multiplying small numbers - numbers less than 1000.
        When you are given two small numbers, multiply them and return the result/
    ''',
    llm_config=llm_config_mini,
    code_execution_config=False,
    human_input_mode="NEVER",
)
    


[autogen.oai.client: 05-21 19:43:41] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [7]:
large_muliplier_agent = autogen.AssistantAgent(
    name = "Large Multiplier Agent",
    system_message = "For multiplying numbers, only use the functions you have been provided with",
    llm_config = llm_config_mini
)

[autogen.oai.client: 05-03 00:05:23] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [8]:
from autogen import GroupChat
group_chat = GroupChat(
    agents = [small_multiplier_agent,large_muliplier_agent],
    messages=[],
    max_round=2,
    speaker_selection_method="auto",
    allow_repeat_speaker= False
)


In [9]:
from autogen import GroupChatManager
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config =llm_config_mini
)

[autogen.oai.client: 05-03 00:05:29] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [10]:
@group_chat_manager.register_for_llm(description="A calculator that does multiplications of large numbers")
@large_muliplier_agent.register_for_execution(name="calculator")
def multiply_large_numbers(num1: float, num2:float) -> float:
    '''
    Returns the sum of two large numbers. 

    Parameters:

    - num1: First number to be multiplied
    - num2: Second number to be multiplied.

    Returns:
    The Product of the two numbers.
    '''

    return num1 * num2

[autogen.oai.client: 05-03 00:05:31] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [11]:

chat_result = user_agent.initiate_chat(group_chat_manager,
                                       message =  "I want to multiply 5 with 2",
                                       summary_method="reflection_with_llm")

user_agent (to chat_manager):

I want to multiply 5 with 2

--------------------------------------------------------------------------------
[autogen.oai.client: 05-03 00:05:51] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.

Next speaker: Small_Multiplier_Agent

Small_Multiplier_Agent (to chat_manager):

The result of multiplying 5 with 2 is 10.

--------------------------------------------------------------------------------


In [13]:
group_chat_manager.last_speaker.name

'Small_Multiplier_Agent'

### Simple multiplier with Open AI Assistant

In [21]:
from typing import Annotated, Literal

import requests

import autogen
from autogen import UserProxyAgent
from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent
from autogen.function_utils import get_function_schema

In [22]:
def multiply_large_numbers(num1: float, num2:float) -> float:
    '''
    Returns the sum of two large numbers. 

    Parameters:

    - num1: First number to be multiplied
    - num2: Second number to be multiplied.

    Returns:
    The Product of the two numbers.
    '''

    return num1 * num2

In [23]:
#Assistant API Tool Schema for multiply_large_numbers
multiply_large_numbers_schema = get_function_schema(
    multiply_large_numbers,
    name = "multiply_large_numbers",
    description = "Simple calculator that can be used for multiplying large numbers, especially when either number is greater than 1000"

)

In [24]:
config_list = autogen.config_list_from_dotenv(
    dotenv_file_path=".env",
    model_api_key_map={
        "gpt-4o": "OPENAI_API_KEY", 
        "gpt-4o-mini": "OPENAI_API_KEY"
    },
    filter_dict={"model":["gpt-4o-mini"]}
)

In [25]:
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "multiplication",
        "use_docker": False},
    
)

In [26]:
multiplier_agent = GPTAssistantAgent(
    name = "simple_calculator",
    instructions = """
    Your role is to multiply the numbers that are given to you. Do the following when you receive a request.

    1. If the numbers are small (less than 1000), return the product.
    2. If they are large, use the `multiply_large_numbers` function to calculate the product and return it.
    """,
    overwrite_instructions=True,
    overwrite_tools=True,
    llm_config= {
        "config_list":config_list,
        "tools":[multiply_large_numbers_schema]

    }
)

[autogen.oai.client: 05-21 19:41:32] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 05-21 19:41:32] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [27]:
multiplier_agent.register_function(
    function_map={
        "multiply_large_numbers":multiply_large_numbers
    }
)

In [28]:
chat_result = user_proxy.initiate_chat(multiplier_agent,
                                       message = " I want to multiply 2456456 with 34343",
                                       summary_method="last_msg",
                                       max_turns = 1)

user_proxy (to simple_calculator):

 I want to multiply 2456456 with 34343

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION multiply_large_numbers...


INFO:autogen.agentchat.contrib.gpt_assistant_agent:Intermediate executing(multiply_large_numbers, Success: True) : 84362068408


simple_calculator (to user_proxy):

The product of 2456456 and 34343 is 84,362,068,408.


--------------------------------------------------------------------------------


In [29]:
chat_result

ChatResult(chat_id=None, chat_history=[{'content': ' I want to multiply 2456456 with 34343', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'The product of 2456456 and 34343 is 84,362,068,408.\n', 'role': 'user', 'name': 'simple_calculator'}], summary='The product of 2456456 and 34343 is 84,362,068,408.\n', cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=[])

# Try Tool Use with Transaction Generation Agent 

## Function for Trxn Generation

In [16]:
from typing import List, Literal
from typing_extensions import Annotated

import numpy as np
from datetime import datetime, timedelta
import random

Channels_allowed = Literal["Wire","Cash","Check"]
def generate_transactions(
        Originator_Name:Annotated[str, "Entity or Customer originating the transactions"],
        Originator_Account_ID:Annotated[str, "Account  of Entity or Customer originating the transactions"],
        Originator_Customer_ID:Annotated[str, "Customer ID of Entity or Customer originating the transactions"],
        Beneficiary_Name:Annotated[str, "Customer ID of Entity or Customer  receiving the transactions"], 
        Beneficiary_Account_ID:Annotated[str, "Account of Entity or Customer  receiving the transactions"],
        Beneficiary_Customer_ID:Annotated[str, "Customer ID of Entity or Customer receiving the transactions"],
        Trxn_Channel:Annotated[List[Channels_allowed], "Transaction Channels used to make the transactions."],
        Start_Date:Annotated[str, "Date on which the first transaction was made"], 
        End_Date:Annotated[str, "Date on which the last transaction was made"],
        Min_Ind_Trxn_Amt:Annotated[float, "The smallest transaction"],
        Max_Ind_Trxn_Amt:Annotated[float, "The largest transaction"],
        Branch_or_ATM_Location:Annotated[str, "The location where transaction was originated or received"],
        N_transactions:Annotated[int, "The number of transactions made between the Originator and Beneficary"]):
    
    Start_Date = datetime.strptime(Start_Date,"%Y-%m-%d")
    End_Date = datetime.strptime(End_Date,"%Y-%m-%d")
    trxns = {} #Dictionary to hold transactions
    trxn_channels = random.choices(Trxn_Channel, k = N_transactions)
    
    sample_deltas  =  random.choices(range((End_Date - Start_Date).days),k = N_transactions) #Get random number of days to be added to get new dates
    trxn_dates = [   Start_Date + timedelta(delta) for delta in sample_deltas] # TO DO: Add start and end date to the list
    #Convert back to string
    trxn_dates = [trxn_date.strftime("%Y-%m-%d") for trxn_date in trxn_dates]
    trxn_amounts = np.round(np.random.uniform(low=Min_Ind_Trxn_Amt,high=Max_Ind_Trxn_Amt,size = N_transactions),2)

    for i in range(N_transactions):
        trxns[f"Trxn_{i+1}"] = {"Originator_Name": Originator_Name , "Originator_Account_ID": Originator_Account_ID,"Originator_Customer_ID": Originator_Customer_ID,
                            "Beneficiary_Name": Beneficiary_Name, "Beneficiary_Account_ID": Beneficiary_Account_ID,"Beneficiary_Customer_ID": Beneficiary_Customer_ID,
                             "Trxn_Channel": trxn_channels[i], "Trxn_Date": trxn_dates[i], "Trxn_Amount":trxn_amounts[i],
                              "Branch_or_ATM_Location": Branch_or_ATM_Location }

    return trxns

In [17]:
generate_transactions(Originator_Name ="Govind" ,Originator_Account_ID = "001",Originator_Customer_ID = "001",
                          Beneficiary_Name = "Neeraja", Beneficiary_Account_ID = "002", Beneficiary_Customer_ID = "002",
                          Trxn_Channel = ["Wire","Check"],Start_Date = "2024-01-01", End_Date = "2024-06-01",Min_Ind_Trxn_Amt = 10000,Max_Ind_Trxn_Amt = 50000,
                          Branch_or_ATM_Location = "Missing", N_transactions =1)

{'Trxn_1': {'Originator_Name': 'Govind',
  'Originator_Account_ID': '001',
  'Originator_Customer_ID': '001',
  'Beneficiary_Name': 'Neeraja',
  'Beneficiary_Account_ID': '002',
  'Beneficiary_Customer_ID': '002',
  'Trxn_Channel': 'Check',
  'Trxn_Date': '2024-01-15',
  'Trxn_Amount': 41681.03,
  'Branch_or_ATM_Location': 'Missing'}}

In [199]:

message1 = '''
{
 "Bank_to_Cust_to_Acct": {
  "Aussie_Bank": {
    "Jennifer Doe": [
      "981012345"
    ]
  },
  "Dummy_Bank": {
    "John Doe": [
      "12345678910"
    ]
  }
},

  "Customers_to_Accts": {
    "John Doe": [
      "12345678910"
    ],
    "Jennifer Doe": [
      "981012345"
    ]
  },

  "FIs_to_Accts": {
    "Aussie Bank": [
      "981012345"
    ],
    "Dummy Bank": [
      "12345678910"
    ]
  },

"Narrative" : {
    "12345678910": "John Doe sent 20 wires between January 17, 2003, and March 21, 2003 to Jennifer Doe. The amount of the wires varied between $3000 and $6000 "
}

}

'''



message2 = '''
{
 'Bank_to_Cust_to_Acct': {
  'Aussie_Bank': {
    'Jennifer_Doe': [
      '981012345'
    ]
  },
  'Dummy_Bank': {
    'John_Doe': [
      '12345678910'
    ]
  }
},

  'Customers_to_Accts': {
    'John_Doe': [
      '12345678910'
    ],
    'Jennifer_Doe': [
      '981012345'
    ]
  },

  'FIs_to_Accts': {
    'Aussie_Bank': [
      '981012345'
    ],
    'Dummy_Bank': [
      '12345678910'
    ]
  },

'Narrative' : {
    '12345678910': 'John_Doe sent 3 wires on Jan 1-2003, Jan 2-2003 and on Jan 3-2003  to Jennifer_Doe. The amount of the wires were $3456, $2457 and $6347 '
}

}

'''

## Trxn Generation Agent with Open AI Assistant Tool

In [18]:
#Assistant API Tool Schema for Trxn Generation
generate_transactions_schema = get_function_schema(
    generate_transactions,
    name = "generate_transactions",
    description = " A function for generating transactions when a large number of transactions have to be synthesizes"

)

NameError: name 'get_function_schema' is not defined

In [15]:
trxn_generation_agent_gpt = GPTAssistantAgent(
    name = "trxn_generation_agent_gpt",
    instructions = """
    You are an AI Agent tasked with synthesizing transactions from a narrative you are provided. Your strength is synthesizing transactions when a larger number of transactions
    are made between two entities  when a range of trxn amounts and dates are specified. When invoked, use the provided function to generate transactions.

    You will have the following four pieces of information.

    
    1) A JSON called Narrative with Account IDs as key  and  narratives potentially describing transactions made by that account as the values. </n>
    2) The mapping between Individuals and Organizations referenced in the narrative to Account IDs, given by the dictionary Customers_to_Accts. Note thataccount IDs starting with the prefix "Dummy_" 
    are placeholders for account IDs that have not been identified in the narrative. </n>
    3) The mapping between Financial Institutions to Account IDs given by the dictionary FIs_to_Accts. If the Financial institution for certain accounts are not specified in the narrative,
      a dummy financial institution "Dummy_Bank" will be used for them.</n>
    4) A mapping between Financial institution, Customer IDs and Accounts owned by those Customer IDs given by the dictionary FI_to_Cust_to_Acct
    
    For the narrative corresponding  to each account ID, do the following:

    Step 1) Identify the number of transactions described in the narrative. Note this in a scratch pad for verification later.

    
    Step 2) Identify the transaction channels or product used. This could be Wire, Cash, Checks or something similar. If this is missing, you can make a reasonable assumption.

    Step 3) Identify the Beneficiary Account ID, the Customer ID corresponding to the account, the beneficiary_Name which is the Individual Or Organization which is the 
    Beneficiary of the transaction. Note that for cash transactions, if only information on the originator or benficary are available, you can assume the originator and beneficiary are the same.

    Step 4) Identify the Originator Account ID, the Customer ID corresponding to the account, the Originating_Name which is the Individual Or Organization originating
    the transaction.

    Step 5) Identify the first and last dates the transactions. If this is missing, you can make reasonable assumptions.

    Step 6) Identify the minimum and maximum amount of the transactions.

    Step 7) If this is a Cash or Check or similar transaction, determine the Branches or ATM Locations where the transaction was conducted. If this is not specified, indicate it as "Missing"

    Ensure the following are extracted.

    -Originator_Name
    -Originator_Account_ID
    -Originator_Customer_ID
    -Beneficiary_Name
    -Beneficiary_Account_ID
    -Beneficiary_Customer_ID
    -Trxn_Channel or List of Trxn Channels
    -First Trxn_Date and Last Trxn Date in yyyy-mm-dd format
    -Min Trxn_Amount and Max Trxn_Amount
    -Branch or ATM Location or List of such locations

    If any information is missing and can't be reasonably assumed, use an empty string "" to indicate it is missing.

    Step 8) Use the function `generate_transactions` that returns a dictionary with the specified number of transactions and attributes.

    An example is given below, demarcated by the delimiter ----..

    ----

    Narrative = {"345723": "John transferred a total of $100,000 from Acct #345723 at Bank of America  to Jill between Jan 1, 2024 and July 4,2024.
                             There were 10 trns, each transactions was $10,000"}

                  Customers_to_Accts = {"John":["345723"],"Jill":["Dummy_12345"],"Acme Inc":"98765"}
                  FIs_to_Accts = {"Bank of America":["345723","98765"],"Chase Bank":["Dummy_12345"]}
                  FI_to_Cust_to_Acct = {"Bank of America": {"CUST_001": ["345723"], "CUST_002":["98765"]}, "Chase Bank": {"CUST_003":["Dummy_12345"]}}

    
   Step 1) Identify the number of transactions described in the narrative. Note this is a scratch pad for verification later.
      There are three trxns described in the narrative. A $5000 deposit, A $3000 transfer and a $1000 transfer.

   Step 2) Identify the transaction channels or product used.
      The first $5000 deposit is Cash. The second $3000 transfer is a Wire. The third $1,000 tranfer is a check.


   Step 3) The Beneficary of the trxns is Jill.The Beneficary_Name is Jill. From Customers_to_Accts, it is noted that Jill's account is Dummy_12345. From FIs_to_Accts, this account is at Chase Bank.
           From FI_to_Cust_to_Acct, the Customer ID corresponding to this account is CUST_003
   

   Step 4) The Originator of the trxns is John. rom the narrative, the Originator name is John and the originating account is 345723. From FIs_to_Accts, this account is at Bank of America.
           From FI_to_Cust_to_Acct, the Customer ID corresponding to this account is CUST_001
           
   Step 5) The first transaction was made on Jan 1,2024 and last transaction was made on July 4, 2024.

   Step 6) The mininmum and maximum transaction amount is $10,000

   Step 7) Since the transactions are Wire transactions, Branch or ATM Location can be considered Missing. All the required attributes have been gathered.

   Step 8) Call the function generatr_transactions with the collected arguments. Return results as JSON only.

    ----


    """,
    overwrite_instructions=True,
    overwrite_tools=True,
    llm_config= {
        "config_list":config_list,
        "tools":[generate_transactions_schema]

    }
)

NameError: name 'GPTAssistantAgent' is not defined

In [207]:
trxn_generation_agent_gpt.register_function(
    function_map={
        "generate_transactions":generate_transactions
    }
)

In [93]:
user_proxy_agent = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "trxn_generation",
        "use_docker": False},
    
)

In [95]:
chat_result = user_proxy.initiate_chat(trxn_generation_agent_gpt,
                                       message = message,
                                       summary_method="reflection_with_llm",
                                       max_turns = 1)

user_proxy (to trxn_generation_agent_gpt):


{
 "Bank_to_Cust_to_Acct": {
  "Aussie Bank": {
    "Jennifer Doe": [
      "981012345"
    ]
  },
  "Dummy_Bank": {
    "John Doe": [
      "12345678910"
    ]
  }
},

  "Customers_to_Accts": {
    "John Doe": [
      "12345678910"
    ],
    "Jennifer Doe": [
      "981012345"
    ]
  },

  "FIs_to_Accts": {
    "Aussie Bank": [
      "981012345"
    ],
    "Dummy_Bank": [
      "12345678910"
    ]
  },

"Narrative" : {
    "12345678910": "John Doe sent 20 wires between January 17, 2003, and March 21, 2003 to Jennifer Doe. The amount of the wires varied between $3000 and $6000 "
}

}



--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION generate_transactions...


INFO:autogen.agentchat.contrib.gpt_assistant_agent:Intermediate executing(generate_transactions, Success: True) : {'Trxn_1': {'Originator_Name': 'John Doe', 'Originator_Account_ID': '12345678910', 'Originator_Customer_ID': 'CUST_002', 'Beneficiary_Name': 'Jennifer Doe', 'Beneficiary_Account_ID': '981012345', 'Beneficiary_Customer_ID': 'CUST_001', 'Trxn_Channel': 'Wire', 'Trxn_Date': '2003-03-10', 'Trxn_Amount': 4689.87, 'Branch_or_ATM_Location': ''}, 'Trxn_2': {'Originator_Name': 'John Doe', 'Originator_Account_ID': '12345678910', 'Originator_Customer_ID': 'CUST_002', 'Beneficiary_Name': 'Jennifer Doe', 'Beneficiary_Account_ID': '981012345', 'Beneficiary_Customer_ID': 'CUST_001', 'Trxn_Channel': 'Wire', 'Trxn_Date': '2003-02-24', 'Trxn_Amount': 4692.46, 'Branch_or_ATM_Location': ''}, 'Trxn_3': {'Originator_Name': 'John Doe', 'Originator_Account_ID': '12345678910', 'Originator_Customer_ID': 'CUST_002', 'Beneficiary_Name': 'Jennifer Doe', 'Beneficiary_Account_ID': '981012345', 'Beneficia

trxn_generation_agent_gpt (to user_proxy):

```json
{
  "Trxn_1": {
    "Originator_Name": "John Doe",
    "Originator_Account_ID": "12345678910",
    "Originator_Customer_ID": "CUST_002",
    "Beneficiary_Name": "Jennifer Doe",
    "Beneficiary_Account_ID": "981012345",
    "Beneficiary_Customer_ID": "CUST_001",
    "Trxn_Channel": "Wire",
    "Trxn_Date": "2003-03-10",
    "Trxn_Amount": 4689.87,
    "Branch_or_ATM_Location": ""
  },
  "Trxn_2": {
    "Originator_Name": "John Doe",
    "Originator_Account_ID": "12345678910",
    "Originator_Customer_ID": "CUST_002",
    "Beneficiary_Name": "Jennifer Doe",
    "Beneficiary_Account_ID": "981012345",
    "Beneficiary_Customer_ID": "CUST_001",
    "Trxn_Channel": "Wire",
    "Trxn_Date": "2003-02-24",
    "Trxn_Amount": 4692.46,
    "Branch_or_ATM_Location": ""
  },
  "Trxn_3": {
    "Originator_Name": "John Doe",
    "Originator_Account_ID": "12345678910",
    "Originator_Customer_ID": "CUST_002",
    "Beneficiary_Name": "Jennifer Doe",

In [96]:
chat_result.chat_history

[{'content': '\n{\n "Bank_to_Cust_to_Acct": {\n  "Aussie Bank": {\n    "Jennifer Doe": [\n      "981012345"\n    ]\n  },\n  "Dummy_Bank": {\n    "John Doe": [\n      "12345678910"\n    ]\n  }\n},\n\n  "Customers_to_Accts": {\n    "John Doe": [\n      "12345678910"\n    ],\n    "Jennifer Doe": [\n      "981012345"\n    ]\n  },\n\n  "FIs_to_Accts": {\n    "Aussie Bank": [\n      "981012345"\n    ],\n    "Dummy_Bank": [\n      "12345678910"\n    ]\n  },\n\n"Narrative" : {\n    "12345678910": "John Doe sent 20 wires between January 17, 2003, and March 21, 2003 to Jennifer Doe. The amount of the wires varied between $3000 and $6000 "\n}\n\n}\n\n',
  'role': 'assistant',
  'name': 'user_proxy'},
 {'content': '```json\n{\n  "Trxn_1": {\n    "Originator_Name": "John Doe",\n    "Originator_Account_ID": "12345678910",\n    "Originator_Customer_ID": "CUST_002",\n    "Beneficiary_Name": "Jennifer Doe",\n    "Beneficiary_Account_ID": "981012345",\n    "Beneficiary_Customer_ID": "CUST_001",\n    "Tr

In [103]:
chat_result.chat_history[1]['content']

'```json\n{\n  "Trxn_1": {\n    "Originator_Name": "John Doe",\n    "Originator_Account_ID": "12345678910",\n    "Originator_Customer_ID": "CUST_002",\n    "Beneficiary_Name": "Jennifer Doe",\n    "Beneficiary_Account_ID": "981012345",\n    "Beneficiary_Customer_ID": "CUST_001",\n    "Trxn_Channel": "Wire",\n    "Trxn_Date": "2003-03-10",\n    "Trxn_Amount": 4689.87,\n    "Branch_or_ATM_Location": ""\n  },\n  "Trxn_2": {\n    "Originator_Name": "John Doe",\n    "Originator_Account_ID": "12345678910",\n    "Originator_Customer_ID": "CUST_002",\n    "Beneficiary_Name": "Jennifer Doe",\n    "Beneficiary_Account_ID": "981012345",\n    "Beneficiary_Customer_ID": "CUST_001",\n    "Trxn_Channel": "Wire",\n    "Trxn_Date": "2003-02-24",\n    "Trxn_Amount": 4692.46,\n    "Branch_or_ATM_Location": ""\n  },\n  "Trxn_3": {\n    "Originator_Name": "John Doe",\n    "Originator_Account_ID": "12345678910",\n    "Originator_Customer_ID": "CUST_002",\n    "Beneficiary_Name": "Jennifer Doe",\n    "Benefi

In [105]:
#chat_result.summary
cleaned_trxns = chat_result.chat_history[1]['content'].split('```',2)[1]
trxns = cleaned_trxns.strip('```json').strip('```')
print(trxns)


{
  "Trxn_1": {
    "Originator_Name": "John Doe",
    "Originator_Account_ID": "12345678910",
    "Originator_Customer_ID": "CUST_002",
    "Beneficiary_Name": "Jennifer Doe",
    "Beneficiary_Account_ID": "981012345",
    "Beneficiary_Customer_ID": "CUST_001",
    "Trxn_Channel": "Wire",
    "Trxn_Date": "2003-03-10",
    "Trxn_Amount": 4689.87,
    "Branch_or_ATM_Location": ""
  },
  "Trxn_2": {
    "Originator_Name": "John Doe",
    "Originator_Account_ID": "12345678910",
    "Originator_Customer_ID": "CUST_002",
    "Beneficiary_Name": "Jennifer Doe",
    "Beneficiary_Account_ID": "981012345",
    "Beneficiary_Customer_ID": "CUST_001",
    "Trxn_Channel": "Wire",
    "Trxn_Date": "2003-02-24",
    "Trxn_Amount": 4692.46,
    "Branch_or_ATM_Location": ""
  },
  "Trxn_3": {
    "Originator_Name": "John Doe",
    "Originator_Account_ID": "12345678910",
    "Originator_Customer_ID": "CUST_002",
    "Beneficiary_Name": "Jennifer Doe",
    "Beneficiary_Account_ID": "981012345",
    "Be

# Group Chat for Trxn Generation

Use a Group Chat Agent to delegate the task to a pure LLM Agent or an LLM Agent equipped with a calculator.

Note to self: Group chat maager should be able to call right agent based on description of trxn egernation agents

In [203]:
sar_agent = autogen.UserProxyAgent(
    name="sar_agent",
    system_message="Share the SAR extract given so that trxns can be extracted",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "temp",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    human_input_mode="ALWAYS",
)

In [195]:
trxn_generation_agent, trxn_generation_agent_gpt


(<autogen.agentchat.conversable_agent.ConversableAgent at 0x256891fe610>,
 <autogen.agentchat.contrib.gpt_assistant_agent.GPTAssistantAgent at 0x256891fe310>)

In [205]:
groupchat = autogen.GroupChat(agents = [sar_agent,trxn_generation_agent, trxn_generation_agent_gpt],messages=[],max_round=2)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config = llm_config_mini)

[autogen.oai.client: 10-08 20:27:20] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [208]:
chat_results = sar_agent.initiate_chat(
    manager,
    message = message2
    )


INFO:autogen.agentchat.contrib.gpt_assistant_agent:Clearing thread thread_SclfQLTbLaFlzXu81mbaJGxB


sar_agent (to chat_manager):


{
 "Bank_to_Cust_to_Acct": {
  "Aussie_Bank": {
    "Jennifer Doe": [
      "981012345"
    ]
  },
  "Dummy_Bank": {
    "John Doe": [
      "12345678910"
    ]
  }
},

  "Customers_to_Accts": {
    "John Doe": [
      "12345678910"
    ],
    "Jennifer Doe": [
      "981012345"
    ]
  },

  "FIs_to_Accts": {
    "Aussie Bank": [
      "981012345"
    ],
    "Dummy Bank": [
      "12345678910"
    ]
  },

"Narrative" : {
    "12345678910": "John Doe sent 20 wires between January 17, 2003, and March 21, 2003 to Jennifer Doe. The amount of the wires varied between $3000 and $6000 "
}

}



--------------------------------------------------------------------------------
[autogen.oai.client: 10-08 20:28:27] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: trxn_generation_agent_gpt


>>>>>>>> EXECUTING FUNCTION generate_transactions...


INFO:autogen.agentchat.contrib.gpt_assistant_agent:Intermediate executing(generate_transactions, Success: True) : {'Trxn_1': {'Originator_Name': 'John Doe', 'Originator_Account_ID': '12345678910', 'Originator_Customer_ID': 'John Doe', 'Beneficiary_Name': 'Jennifer Doe', 'Beneficiary_Account_ID': '981012345', 'Beneficiary_Customer_ID': 'Jennifer Doe', 'Trxn_Channel': 'Wire', 'Trxn_Date': '2003-03-01', 'Trxn_Amount': 3527.39, 'Branch_or_ATM_Location': 'Missing'}, 'Trxn_2': {'Originator_Name': 'John Doe', 'Originator_Account_ID': '12345678910', 'Originator_Customer_ID': 'John Doe', 'Beneficiary_Name': 'Jennifer Doe', 'Beneficiary_Account_ID': '981012345', 'Beneficiary_Customer_ID': 'Jennifer Doe', 'Trxn_Channel': 'Wire', 'Trxn_Date': '2003-01-24', 'Trxn_Amount': 4341.18, 'Branch_or_ATM_Location': 'Missing'}, 'Trxn_3': {'Originator_Name': 'John Doe', 'Originator_Account_ID': '12345678910', 'Originator_Customer_ID': 'John Doe', 'Beneficiary_Name': 'Jennifer Doe', 'Beneficiary_Account_ID': '

trxn_generation_agent_gpt (to chat_manager):

```json
{
  "Trxn_1": {
    "Originator_Name": "John Doe",
    "Originator_Account_ID": "12345678910",
    "Originator_Customer_ID": "John Doe",
    "Beneficiary_Name": "Jennifer Doe",
    "Beneficiary_Account_ID": "981012345",
    "Beneficiary_Customer_ID": "Jennifer Doe",
    "Trxn_Channel": "Wire",
    "Trxn_Date": "2003-03-01",
    "Trxn_Amount": 3527.39,
    "Branch_or_ATM_Location": "Missing"
  },
  "Trxn_2": {
    "Originator_Name": "John Doe",
    "Originator_Account_ID": "12345678910",
    "Originator_Customer_ID": "John Doe",
    "Beneficiary_Name": "Jennifer Doe",
    "Beneficiary_Account_ID": "981012345",
    "Beneficiary_Customer_ID": "Jennifer Doe",
    "Trxn_Channel": "Wire",
    "Trxn_Date": "2003-01-24",
    "Trxn_Amount": 4341.18,
    "Branch_or_ATM_Location": "Missing"
  },
  "Trxn_3": {
    "Originator_Name": "John Doe",
    "Originator_Account_ID": "12345678910",
    "Originator_Customer_ID": "John Doe",
    "Beneficiar

In [214]:
cleaned_trxns = chat_results.chat_history[1]['content'].split('```',2)[1]
trxns = cleaned_trxns.strip('```json').strip('```')
print(trxns)


{
  "Trxn_1": {
    "Originator_Name": "John Doe",
    "Originator_Account_ID": "12345678910",
    "Originator_Customer_ID": "John Doe",
    "Beneficiary_Name": "Jennifer Doe",
    "Beneficiary_Account_ID": "981012345",
    "Beneficiary_Customer_ID": "Jennifer Doe",
    "Trxn_Channel": "Wire",
    "Trxn_Date": "2003-03-01",
    "Trxn_Amount": 3527.39,
    "Branch_or_ATM_Location": "Missing"
  },
  "Trxn_2": {
    "Originator_Name": "John Doe",
    "Originator_Account_ID": "12345678910",
    "Originator_Customer_ID": "John Doe",
    "Beneficiary_Name": "Jennifer Doe",
    "Beneficiary_Account_ID": "981012345",
    "Beneficiary_Customer_ID": "Jennifer Doe",
    "Trxn_Channel": "Wire",
    "Trxn_Date": "2003-01-24",
    "Trxn_Amount": 4341.18,
    "Branch_or_ATM_Location": "Missing"
  },
  "Trxn_3": {
    "Originator_Name": "John Doe",
    "Originator_Account_ID": "12345678910",
    "Originator_Customer_ID": "John Doe",
    "Beneficiary_Name": "Jennifer Doe",
    "Beneficiary_Account_ID"

In [216]:
chat_results = sar_agent.initiate_chat(
    manager,
    message = message2
    )

INFO:autogen.agentchat.contrib.gpt_assistant_agent:Clearing thread thread_XHpzidvbNcjXhUzqDAklicHG


sar_agent (to chat_manager):


{
 'Bank_to_Cust_to_Acct': {
  'Aussie_Bank': {
    'Jennifer_Doe': [
      '981012345'
    ]
  },
  'Dummy_Bank': {
    'John_Doe': [
      '12345678910'
    ]
  }
},

  'Customers_to_Accts': {
    'John_Doe': [
      '12345678910'
    ],
    'Jennifer_Doe': [
      '981012345'
    ]
  },

  'FIs_to_Accts': {
    'Aussie_Bank': [
      '981012345'
    ],
    'Dummy_Bank': [
      '12345678910'
    ]
  },

'Narrative' : {
    '12345678910': 'John_Doe sent 3 wires on Jan 1-2003, Jan 2-2003 and on Jan 3-2003  to Jennifer_Doe. The amount of the wires were $3456, $2457 and $6347 '
}

}



--------------------------------------------------------------------------------
[autogen.oai.client: 10-08 20:32:04] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: trxn_generation_agent

trxn_generation_agent (to chat_manager):

Step 1) Identify the number of transactions described in the narrative.   
There are three transactions described in the narrative: three wires sent to Jennifer_Doe on January 1, 2, and 3, 2003 for three different amounts.

Step 2) For each transaction, identify the required attributes.

For the first transaction on January 1, 2003:  
- Originator_Name: John_Doe  
- Originator_Account_ID: 12345678910 (from Customers_to_Accts)  
- Originator_Customer_ID: John_Doe (from Customers_to_Accts)  
- Beneficiary_Name: Jennifer_Doe  
- Beneficiary_Account_ID: 981012345 (from Customers_to_Accts for Jennifer)  
- Beneficiary_Customer_ID: Jennifer_Doe (from Customers_to_Accts)  
- Trxn_Channel: Wire  
- Trxn_Date: 2003-01-01  
- Trxn_Amount: 3456  
- Branch or ATM Location: Missing  

For the second transaction on January 2, 2003:  
- Originator_Name: John_Doe  
- Originator_Account_ID: 12345678910  
- Originator_Custom

# Trxns

In [76]:
from autogen import GroupChat

group_chat = GroupChat(
    agents=[entity_extraction_agent, entity_resolution_agent],
    messages=[],
    max_round=2,
    speaker_selection_method = "round_robin",
)

In [ ]:
from autogen import GroupChatManager
help(GroupChatManager)

# Rough Work

## Check GroupChat

In [179]:
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    human_input_mode="TERMINATE",
)
coder = autogen.AssistantAgent(
    name="Coder",
    llm_config=llm_config_mini,
)
pm = autogen.AssistantAgent(
    name="Product_manager",
    system_message="Creative in software product ideas.",
    llm_config=llm_config_mini,
)
groupchat = autogen.GroupChat(agents=[user_proxy, coder, pm], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config_mini)

[autogen.oai.client: 10-08 20:08:30] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 10-08 20:08:30] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 10-08 20:08:30] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [159]:
user_proxy.initiate_chat(
    manager, message="Find a latest paper about gpt-4 on arxiv and find its potential applications in software."
)

User_proxy (to chat_manager):

Find a latest paper about gpt-4 on arxiv and find its potential applications in software.

--------------------------------------------------------------------------------
[autogen.oai.client: 10-08 19:23:25] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: User_proxy


>>>>>>>> USING AUTO REPLY...
User_proxy (to chat_manager):



--------------------------------------------------------------------------------
[autogen.oai.client: 10-08 19:23:26] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: User_proxy


>>>>>>>> USING AUTO REPLY...
User_proxy (to chat_manager):



--------------------------------------------------------------------------------
[autogen.oai.client: 10-08 19:23:27] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: User_proxy


>>>>>>>> USING AUTO REPLY...
User_proxy (to chat_manager):



--------------------------------------------------------------------------------
[autogen.oai.client: 10-08 19:23:27] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: Coder

Coder (to chat_manager):

To find the latest paper about GPT-4 on arXiv and its potential applications in software, I will first retrieve the list of recent papers related to GPT-4 from arXiv. Then I will extract the relevant information regarding its applications in software.

Let's start by fetching the latest papers on GPT-4 from arXiv.

```python
import requests

# Define the arXiv API endpoint for searching
query = 'gpt-4'
url = f'http://export.arxiv.org/api/query?search_query=all:{query}&start=0&max_results=5&sortBy=submittedDate&sortOrder=descending'

# Fetch the data
response = requests.get(url)
data = response.text

print(data)
```

Execute this code to retrieve the latest papers about GPT-4 from arXiv.

--------------------------------------------------------------------------------
[autogen.oai.client: 10-08 19:23:30] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


...


Next speaker: Product_manager

Product_manager (to chat_manager):

The summary document has been successfully created with the latest findings about GPT-4 and their potential software applications. You can find it in the file named **gpt4_paper_summary.txt**.

Here’s a quick recap of the contents:

1. **Recent Papers**:
   - "Contrastive Learning to Improve Retrieval for Real-world Fact Checking"
   - "ProtocoLLM: Automatic Evaluation Framework of LLMs on Domain-Specific Scientific Protocol Formulation Tasks"

2. **Potential Applications**:
   - Development of automated fact-checking tools.
   - Creation of software for automating and validating scientific research protocols.

If you have any further questions or need additional information, feel free to ask!

--------------------------------------------------------------------------------
[autogen.oai.client: 10-08 19:24:03] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model


Next speaker: Product_manager

Product_manager (to chat_manager):

If you have any further tasks or need additional assistance regarding the document or the papers, please feel free to let me know!

--------------------------------------------------------------------------------
[autogen.oai.client: 10-08 19:24:05] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: Product_manager

Product_manager (to chat_manager):

If you have any further tasks or need additional assistance regarding the document or the papers, please feel free to let me know!

--------------------------------------------------------------------------------
[autogen.oai.client: 10-08 19:24:07] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: User_proxy


>>>>>>>> USING AUTO REPLY...
User_proxy (to chat_manager):



--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'Find a latest paper about gpt-4 on arxiv and find its potential applications in software.', 'role': 'assistant', 'name': 'User_proxy'}, {'content': '', 'role': 'assistant', 'name': 'User_proxy'}, {'content': '', 'role': 'assistant', 'name': 'User_proxy'}, {'content': '', 'role': 'assistant', 'name': 'User_proxy'}, {'content': "To find the latest paper about GPT-4 on arXiv and its potential applications in software, I will first retrieve the list of recent papers related to GPT-4 from arXiv. Then I will extract the relevant information regarding its applications in software.\n\nLet's start by fetching the latest papers on GPT-4 from arXiv.\n\n```python\nimport requests\n\n# Define the arXiv API endpoint for searching\nquery = 'gpt-4'\nurl = f'http://export.arxiv.org/api/query?search_query=all:{query}&start=0&max_results=5&sortBy=submittedDate&sortOrder=descending'\n\n# Fetch the data\nresponse = requests.get(url)\ndata = response.text\

In [180]:
user_proxy.initiate_chat(
    manager, message="Find the product of 5 and 2"
)

User_proxy (to chat_manager):

Find the product of 5 and 2

--------------------------------------------------------------------------------
[autogen.oai.client: 10-08 20:09:00] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: Coder

Coder (to chat_manager):

To find the product of 5 and 2, I will provide you with a simple Python code that calculates and prints the result.

```python
# filename: calculate_product.py
product = 5 * 2
print(product)
```

Please save the code in a file named `calculate_product.py` and execute it.

--------------------------------------------------------------------------------
[autogen.oai.client: 10-08 20:09:03] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: User_proxy


>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
User_proxy (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 
10


--------------------------------------------------------------------------------
[autogen.oai.client: 10-08 20:09:03] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: Coder

Coder (to chat_manager):

The product of 5 and 2 is 10. 

TERMINATE

--------------------------------------------------------------------------------
[autogen.oai.client: 10-08 20:09:04] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



Next speaker: Coder

Coder (to chat_manager):

TERMINATE

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'Find the product of 5 and 2', 'role': 'assistant', 'name': 'User_proxy'}, {'content': 'To find the product of 5 and 2, I will provide you with a simple Python code that calculates and prints the result.\n\n```python\n# filename: calculate_product.py\nproduct = 5 * 2\nprint(product)\n```\n\nPlease save the code in a file named `calculate_product.py` and execute it.', 'name': 'Coder', 'role': 'user'}, {'content': 'exitcode: 0 (execution succeeded)\nCode output: \n10\n', 'role': 'assistant', 'name': 'User_proxy'}, {'content': 'The product of 5 and 2 is 10. \n\nTERMINATE', 'name': 'Coder', 'role': 'user'}, {'content': 'TERMINATE', 'name': 'Coder', 'role': 'user'}], summary='', cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=[])

In [1]:
dict1 = {"a":1,"b":2}
dict2 = {"b":2,"a":1}

In [2]:
assert dict1 ==dict2 

In [3]:
from datetime import datetime

# Define the date string and the format it follows
date_string = "January 17, 2003"
date_format = "%B %d, %Y"

# Convert the string to a datetime object
date_object = datetime.strptime(date_string, date_format)

print(date_object) 

2003-01-17 00:00:00


In [5]:
sar1_trxns = {
    1: {"Originator_Name": "John Doe", "Originator_Account_ID": "12345-6789", "Beneficiary_Name": "Kulkutta Building Supply Company","Beneficiary_Acount_ID":"3489728",
        "Trxn_Channel":"Wire", "Trxn_Date": datetime(2003,1,17), "Trxn_Amount":25000, "Branch or ATM Location": "Not Specified"
        },
    2: {"Originator_Name": "John Doe", "Originator_Account_ID": "12345-6789", "Beneficiary_Name": "Kulkutta Building Supply Company","Beneficiary_Acount_ID":"3489728",
        "Trxn_Channel":"Wire", "Trxn_Date": datetime(2003,1,24), "Trxn_Amount":25000, "Branch or ATM Location": "Not Specified"
        }

}

In [6]:
len(sar1_trxns)

2

In [8]:
set([1,2,3]) == set([1,3,2])

True

In [9]:
({"a":1},{"b":2})

({'a': 1}, {'b': 2})

In [10]:
import pandas as pd

In [24]:
temp = pd.DataFrame(
    [{"Originator_Name": "John Doe", "Originator_Account_ID": "12345-6789", "Beneficiary_Name": "Kulkutta Building Supply Company","Beneficiary_Account_ID":"3489728",
        "Trxn_Channel":"Wire", "Trxn_Date": datetime(2003,1,17), "Trxn_Amount":25000, "Branch or ATM Location": "Not Specified"
        },
    {"Originator_Name": "John Doe", "Originator_Account_ID": "12345-6789", "Beneficiary_Name": "Kulkutta Building Supply Company","Beneficiary_Account_ID":"3489728",
        "Trxn_Channel":"Wire", "Trxn_Date": datetime(2003,1,24), "Trxn_Amount":25000, "Branch or ATM Location": "Not Specified"
        }]
)

In [18]:
print(temp)

  Originator_Name Originator_Account_ID                  Beneficiary_Name  \
0        John Doe            12345-6789  Kulkutta Building Supply Company   
1        John Doe            12345-6789  Kulkutta Building Supply Company   

  Beneficiary_Acount_ID Trxn_Channel  Trxn_Date  Trxn_Amount  \
0               3489728         Wire 2003-01-17        25000   
1               3489728         Wire 2003-01-24        25000   

  Branch or ATM Location  
0          Not Specified  
1          Not Specified  


In [36]:
temp[(temp["Originator_Account_ID"] == "12345-6789") & (temp["Beneficiary_Account_ID"]== "3489728")].shape[0]

2

In [37]:
sum(temp["Trxn_Amount"])

50000

In [34]:
condition1 = temp["Originator_Account_ID"] == "12345-6789" 

In [33]:
condition2 = temp["Beneficiary_Account_ID"]== "3489728"

In [35]:
condition1 & condition2

0    True
1    True
dtype: bool

In [43]:
set(["Wire"]) ==  set(temp["Trxn_Channel"])

True

In [44]:
1.1/(2.1)


0.5238095238095238

In [45]:
9.9/(1+9.9)


0.908256880733945

In [13]:
{ 
    "Entities" : {"Individuals": ["John", "Jill"],"Organizations":["Acme Inc"],"Financial Institution":["Chase"," Bank of America", "Citibank"]},
    "Account IDs": ["345723","98765","Dummy_12345"],
    "Entity_to_Acct_1" : {"John":["345723"],"Jill":["Dummy_12345"],"Acme Inc":"98765"},
    "Entity_to_Acct_2" : {"Bank of America":["345723"],"Chase Bank":["Dummy_12345"],"Citibank":"98765"}

}

{'Entities': {'Individuals': ['John', 'Jill'],
  'Organizations': ['Acme Inc'],
  'Financial Institution': ['Chase', ' Bank of America', 'Citibank']},
 'Account IDs': ['345723', '98765', 'Dummy_12345'],
 'Entity_to_Acct_1': {'John': ['345723'],
  'Jill': ['Dummy_12345'],
  'Acme Inc': '98765'},
 'Entity_to_Acct_2': {'Bank of America': ['345723'],
  'Chase Bank': ['Dummy_12345'],
  'Citibank': '98765'}}